In [4]:
!pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.2 MB/s eta 0:00:00


In [82]:
# product_definitions.py
"""
Contains classes for defining the static properties of various financial products.
Handles date string parsing in from_dict methods and __init__.
ConvertibleBondStaticBase no longer holds dynamic market parameters like vol, div_yield, etc.
These will be passed to the pricer.
Ensures index_stub has a default if not provided.
"""
import QuantLib as ql
from datetime import date, datetime
import abc
import numpy as np

def _parse_date_input(date_input):
    """Helper to parse date input which could be date object or ISO string."""
    if isinstance(date_input, datetime):
        return date_input.date()
    if isinstance(date_input, date):
        return date_input
    if isinstance(date_input, str):
        try:
            return date.fromisoformat(date_input)
        except ValueError:
            try:
                dt_obj = datetime.fromisoformat(date_input.replace('Z', '+00:00'))
                return dt_obj.date()
            except ValueError:
                raise ValueError(f"Invalid date string format: '{date_input}'. Expected YYYY-MM-DD or ISO datetime.")
    raise TypeError(f"Unsupported date input type: {type(date_input)}. Value: {date_input}")

def _serialize_date_list(date_list):
    if date_list is None: return None
    return [_parse_date_input(d).isoformat() if d else None for d in date_list]

def _parse_date_list(date_input_list):
    if date_input_list is None: return []
    return [_parse_date_input(d_str) if d_str else None for d_str in date_input_list]


class ProductStaticBase(abc.ABC):
    def __init__(self, valuation_date):
        self.valuation_date_py: date = _parse_date_input(valuation_date)

    @classmethod
    @abc.abstractmethod
    def from_dict(cls, params: dict) -> 'ProductStaticBase':
        pass

    @abc.abstractmethod
    def to_dict(self) -> dict:
        pass


class QuantLibBondStaticBase(ProductStaticBase):
    def __init__(
        self,
        valuation_date, maturity_date, coupon_rate: float,
        face_value: float = 100.0, freq: int = 2,
        calendar = None, day_count = None,
        business_convention: int = None, settlement_days: int = 0,
        currency: str = "USD", index_stub: str = "GENERIC_IR"
    ):
        super().__init__(valuation_date)
        self.maturity_date_py: date = _parse_date_input(maturity_date)
        self.coupon_rate: float = float(coupon_rate)
        self.face_value: float = float(face_value)
        self.freq: int = int(freq)
        self.settlement_days: int = int(settlement_days)
        self.currency: str = currency
        self.index_stub: str = index_stub if index_stub and index_stub.strip() else "GENERIC_IR"

        if isinstance(calendar, str):
            if calendar.lower() == "target": self.calendar_ql = ql.TARGET()
            elif calendar.lower() == "us_federalreserve": self.calendar_ql = ql.UnitedStates(ql.UnitedStates.FederalReserve)
            else: self.calendar_ql = ql.NullCalendar()
        elif isinstance(calendar, ql.Calendar): self.calendar_ql = calendar
        else:
            if self.currency.upper() == "USD": self.calendar_ql = ql.UnitedStates(ql.UnitedStates.FederalReserve)
            elif self.currency.upper() == "EUR": self.calendar_ql = ql.TARGET()
            else: self.calendar_ql = ql.TARGET()

        if isinstance(day_count, str):
            if day_count.lower() == "actualactualisda": self.day_count_ql = ql.ActualActual(ql.ActualActual.ISDA)
            elif day_count.lower() == "actual360": self.day_count_ql = ql.Actual360()
            elif day_count.lower() == "thirty360": self.day_count_ql = ql.Thirty360(ql.Thirty360.USA)
            else: self.day_count_ql = ql.Actual365Fixed()
        elif isinstance(day_count, ql.DayCounter): self.day_count_ql = day_count
        else: self.day_count_ql = ql.ActualActual(ql.ActualActual.ISDA)

        self.business_convention_ql: int = business_convention if business_convention is not None else ql.Following

        self.ql_valuation_date: ql.Date = ql.Date(self.valuation_date_py.day, self.valuation_date_py.month, self.valuation_date_py.year)
        ql.Settings.instance().evaluationDate = self.ql_valuation_date
        self.ql_maturity_date: ql.Date = ql.Date(self.maturity_date_py.day, self.maturity_date_py.month, self.maturity_date_py.year)
        self.issue_date_ql: ql.Date = self.ql_valuation_date

        months_in_period = int(12 / self.freq)
        self.schedule: ql.Schedule = ql.Schedule(
            self.issue_date_ql, self.ql_maturity_date,
            ql.Period(months_in_period, ql.Months), self.calendar_ql,
            self.business_convention_ql, self.business_convention_ql,
            ql.DateGeneration.Forward, False)
        self.bond: ql.Bond = ql.FixedRateBond(
            self.settlement_days, self.face_value, self.schedule,
            [self.coupon_rate], self.day_count_ql,
            self.business_convention_ql, self.face_value)

    @classmethod
    def from_dict(cls, params: dict) -> 'QuantLibBondStaticBase':
        return cls(
            valuation_date=params['valuation_date'],
            maturity_date=params['maturity_date'],
            coupon_rate=float(params['coupon_rate']),
            face_value=float(params.get('face_value', 100.0)),
            freq=int(params.get('freq', 2)),
            calendar=params.get('calendar'), day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            settlement_days=int(params.get('settlement_days', 0)),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub', "GENERIC_IR")
        )

    def to_dict(self) -> dict:
        return {
            'product_type': 'VanillaBond',
            'valuation_date': self.valuation_date_py.isoformat(),
            'maturity_date': self.maturity_date_py.isoformat(),
            'coupon_rate': self.coupon_rate, 'face_value': self.face_value,
            'freq': self.freq, 'settlement_days': self.settlement_days,
            'currency': self.currency, 'index_stub': self.index_stub,
            'calendar': self.calendar_ql.name() if self.calendar_ql else None,
            'day_count': self.day_count_ql.name() if self.day_count_ql else None,
        }


class CallableBondStaticBase(QuantLibBondStaticBase):
    def __init__(
        self, valuation_date, maturity_date, coupon_rate: float,
        call_dates: list, call_prices: list[float], face_value: float = 100.0,
        freq: int = 2, calendar = None, day_count = None,
        business_convention: int = None, settlement_days: int = 0,
        currency: str = "USD", index_stub: str = "GENERIC_IR" ):
        super().__init__(valuation_date, maturity_date, coupon_rate, face_value, freq,
                         calendar, day_count, business_convention, settlement_days, currency, index_stub)
        self.call_dates_py: list[date] = _parse_date_list(call_dates)
        self.call_prices_py: list[float] = [float(p) for p in call_prices]
        self.call_schedule: ql.CallabilitySchedule = ql.CallabilitySchedule()
        if self.call_dates_py:
            for cd_py, cp in zip(self.call_dates_py, self.call_prices_py):
                if cd_py is None: continue
                ql_cd = ql.Date(cd_py.day, cd_py.month, cd_py.year)
                call = ql.Callability(ql.BondPrice(cp, ql.BondPrice.Clean), ql.Callability.Call, ql_cd)
                self.call_schedule.push_back(call)
        self.bond: ql.CallableFixedRateBond = ql.CallableFixedRateBond(
            self.settlement_days, self.face_value, self.schedule, [self.coupon_rate],
            self.day_count_ql, self.business_convention_ql, self.face_value,
            self.issue_date_ql, self.call_schedule)

    @classmethod
    def from_dict(cls, params: dict) -> 'CallableBondStaticBase':
        return cls(
            valuation_date=params['valuation_date'], maturity_date=params['maturity_date'],
            coupon_rate=float(params['coupon_rate']),
            call_dates=params.get('call_dates', []), call_prices=params.get('call_prices',[]),
            face_value=float(params.get('face_value', 100.0)), freq=int(params.get('freq', 2)),
            calendar=params.get('calendar'), day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            settlement_days=int(params.get('settlement_days', 0)),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub', "GENERIC_IR"))

    def to_dict(self) -> dict:
      base = super().to_dict(); base.update({
          'product_type': 'CallableBond',
          'call_dates': _serialize_date_list(self.call_dates_py), 'call_prices': self.call_prices_py
      }); return base


class ConvertibleBondStaticBase(QuantLibBondStaticBase):
    def __init__(
        self, valuation_date, issue_date, maturity_date, coupon_rate: float,
        conversion_ratio: float, face_value: float = 100.0, freq: int = 2,
        settlement_days: int = 0, calendar = None, day_count = None,
        business_convention: int = None, exercise_type: str = 'EuropeanAtMaturity',
        currency: str = "USD", index_stub: str = "GENERIC_IR", underlying_symbol: str = None ):
        super().__init__(valuation_date, maturity_date, coupon_rate, face_value, freq, calendar,
                         day_count, business_convention, settlement_days, currency, index_stub)
        self.issue_date_py: date = _parse_date_input(issue_date)
        self.issue_date_ql = ql.Date(self.issue_date_py.day, self.issue_date_py.month, self.issue_date_py.year)

        months_in_period = int(12 / self.freq)
        self.schedule = ql.Schedule(
            self.issue_date_ql, self.ql_maturity_date, ql.Period(months_in_period, ql.Months),
            self.calendar_ql, self.business_convention_ql, self.business_convention_ql,
            ql.DateGeneration.Forward, False)

        self.conversion_ratio: float = float(conversion_ratio)
        self.exercise_type_str: str = exercise_type
        self.underlying_symbol: str = underlying_symbol

        if self.exercise_type_str == 'EuropeanAtMaturity':
            self.exercise: ql.Exercise = ql.EuropeanExercise(self.ql_maturity_date)
        else: raise ValueError(f"Unsupported exercise type: {self.exercise_type_str}")
        self.convertible_call_schedule: ql.CallabilitySchedule = ql.CallabilitySchedule()
        self.bond: ql.ConvertibleFixedCouponBond = ql.ConvertibleFixedCouponBond(
            self.exercise, self.conversion_ratio, self.convertible_call_schedule,
            self.issue_date_ql, self.settlement_days, [self.coupon_rate],
            self.day_count_ql, self.schedule, self.face_value)

    @classmethod
    def from_dict(cls, params: dict) -> 'ConvertibleBondStaticBase':
        return cls(
            valuation_date=params['valuation_date'], issue_date=params['issue_date'],
            maturity_date=params['maturity_date'], coupon_rate=float(params['coupon_rate']),
            conversion_ratio=float(params['conversion_ratio']),
            face_value=float(params.get('face_value', 100.0)), freq=int(params.get('freq', 2)),
            settlement_days=int(params.get('settlement_days', 0)),
            calendar=params.get('calendar'), day_count=params.get('day_count'),
            business_convention=params.get('business_convention'),
            exercise_type=params.get('exercise_type', 'EuropeanAtMaturity'),
            currency=params.get('currency', "USD"),
            index_stub=params.get('index_stub', "GENERIC_IR"),
            underlying_symbol=params.get('underlying_symbol'))

    def to_dict(self) -> dict:
        base = super().to_dict(); base.update({
            'product_type': 'ConvertibleBond', 'issue_date': self.issue_date_py.isoformat(),
            'conversion_ratio': self.conversion_ratio, 'exercise_type': self.exercise_type_str,
            'underlying_symbol': self.underlying_symbol
        }); return base


class EuropeanOptionStatic(ProductStaticBase):
    def __init__(self, valuation_date, expiry_date, strike_price: float, option_type: str,
                 day_count_convention = None, currency: str = "USD", underlying_symbol: str = None):
        super().__init__(valuation_date)
        self.expiry_date_py: date = _parse_date_input(expiry_date)
        self.strike_price: float = float(strike_price)
        self.currency: str = currency
        self.underlying_symbol: str = underlying_symbol

        if option_type.lower() not in ['call', 'put']: raise ValueError("Option type must be 'call' or 'put'")
        self.option_type: str = option_type.lower()

        ql_valuation_date = ql.Date(self.valuation_date_py.day, self.valuation_date_py.month, self.valuation_date_py.year)
        ql.Settings.instance().evaluationDate = ql_valuation_date
        ql_expiry_date = ql.Date(self.expiry_date_py.day, self.expiry_date_py.month, self.expiry_date_py.year)

        if isinstance(day_count_convention, str):
            if day_count_convention.lower() == "actual365fixed": self.day_count_convention_ql = ql.Actual365Fixed()
            elif day_count_convention.lower() == "actual360": self.day_count_convention_ql = ql.Actual360()
            else: self.day_count_convention_ql = ql.Actual365Fixed()
        elif isinstance(day_count_convention, ql.DayCounter): self.day_count_convention_ql = day_count_convention
        else: self.day_count_convention_ql = ql.Actual365Fixed()

        self.time_to_expiry: float = self.day_count_convention_ql.yearFraction(ql_valuation_date, ql_expiry_date)
        if self.time_to_expiry < 0: self.time_to_expiry = 0.0

    @classmethod
    def from_dict(cls, params: dict) -> 'EuropeanOptionStatic':
        return cls(
            valuation_date=params['valuation_date'], expiry_date=params['expiry_date'],
            strike_price=float(params['strike_price']), option_type=params['option_type'],
            day_count_convention=params.get('day_count_convention'),
            currency=params.get('currency', "USD"),
            underlying_symbol=params.get('underlying_symbol'))

    def to_dict(self) -> dict:
        return {
            'product_type': 'EuropeanOption',
            'valuation_date': self.valuation_date_py.isoformat(),
            'expiry_date': self.expiry_date_py.isoformat(),
            'strike_price': self.strike_price, 'option_type': self.option_type,
            'day_count_convention': self.day_count_convention_ql.name() if self.day_count_convention_ql else None,
            'currency': self.currency, 'underlying_symbol': self.underlying_symbol
        }


In [94]:
# quanlib_custom_serializer.py
import json

def custom_json_serializer(obj):
    """
    Custom serializer for json.dumps that handles ProductStaticBase instances.
    """
    # Check if the object has a to_dict method
    if hasattr(obj, 'to_dict') and callable(obj.to_dict):
        return obj.to_dict()
    # Convert datetime.date objects to ISO format string
    elif isinstance(obj, date):
        return obj.isoformat()
    # Add checks for other non-serializable types if necessary
    # For now, rely on default JSONEncoder for built-in types
    # If other non-serializable objects (like QuantLib objects) are *not*
    # handled by the to_dict methods, they will still cause errors.
    # Ensure your to_dict methods convert all custom/QL objects to serializable types.
    # For QuantLib objects, you might need to extract simple values (like strings, floats, ints).
    # For example, a ql.Date can be converted to a string or a Python date.
    # ql.Calendar, ql.DayCounter etc. are usually not serializable directly;
    # represent them by name (string) or relevant properties in to_dict.
    elif isinstance(obj, ql.Date):
         return date(obj.year(), obj.month(), obj.day()).isoformat()
    elif isinstance(obj, ql.Calendar):
        return obj.name() # Represent calendar by its name string
    elif isinstance(obj, ql.DayCounter):
        return obj.name() # Represent day counter by its name string
    elif isinstance(obj, ql.Exercise):
        # This depends on the Exercise type. For EuropeanExercise, you might serialize the expiry date.
        if isinstance(obj, ql.EuropeanExercise):
            return {'type': 'EuropeanExercise', 'expiry_date': date(obj.date().year(), obj.date().month(), obj.date().day()).isoformat()}
        else:
             # Handle other QuantLib types or raise a TypeError if not handled
            raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable without custom handling.")
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        # Let the default encoder handle other types
        return json.JSONEncoder.default(None, obj)

In [95]:
# pricers.py
"""
Contains pricer classes for different financial products.
Pricers take a static product definition and market data to calculate a price.
Convertible bond pricer now correctly handles dynamic vs. fixed market parameters.
Ensures is_convertible and is_callable attributes are correctly set.
"""
import QuantLib as ql
import numpy as np
from scipy.stats import norm
import abc

class PricerBase(abc.ABC):
    def __init__(self, product_static: ProductStaticBase):
        self.product_static: ProductStaticBase = product_static
    @abc.abstractmethod
    def price(self, **kwargs) -> np.ndarray: pass

class FastBondPricer(PricerBase):
    def __init__(self, bond_static: QuantLibBondStaticBase):
        super().__init__(bond_static)
        self._gen_cashflows()
    def _gen_cashflows(self):
        bond_def: QuantLibBondStaticBase = self.product_static
        ql_sched, dc, ql_val = bond_def.schedule, bond_def.day_count_ql, bond_def.ql_valuation_date
        coupon_amt = bond_def.face_value * bond_def.coupon_rate / bond_def.freq
        cf_d, cf_t, cf_a = [],[],[]
        for i in range(len(ql_sched)):
            d = ql_sched[i]
            if d <= ql_val: continue
            cf_d.append(d.to_date()); cf_t.append(dc.yearFraction(ql_val, d))
            curr_cf = coupon_amt
            if d == bond_def.ql_maturity_date or d == ql_sched[-1]: curr_cf += bond_def.face_value
            cf_a.append(curr_cf)
        self.cf_dates, self.cf_times, self.cf_amounts = cf_d, np.array(cf_t,float), np.array(cf_a,float)
        valid = self.cf_times > 1e-9
        self.cf_times, self.cf_amounts = self.cf_times[valid], self.cf_amounts[valid]
        self.cf_dates = [dt for i, dt in enumerate(self.cf_dates) if valid[i]]
    def price(self, pillar_times: np.ndarray, market_scenario_data: np.ndarray, **kwargs) -> np.ndarray:
        rates = market_scenario_data
        num_p = len(pillar_times)
        if rates.ndim == 2 and rates.shape[1] > num_p: rates = rates[:, :num_p]
        elif rates.ndim == 1 and len(rates) > num_p: rates = rates[:num_p]
        if not self.cf_times.size: return np.zeros(rates.shape[0]) if rates.ndim == 2 else np.array([0.0])
        if rates.ndim == 1:
            r = np.interp(self.cf_times, pillar_times, rates); dfs = np.exp(-r*self.cf_times)
            return np.array([float(self.cf_amounts.dot(dfs))])
        r_mat = np.array([np.interp(self.cf_times, pillar_times, sr) for sr in rates])
        dfs_mat = np.exp(-r_mat * self.cf_times[None, :]); return dfs_mat.dot(self.cf_amounts)

class QuantLibBondPricer(PricerBase):
    def __init__(self, bond_static: QuantLibBondStaticBase, method: str = 'discount',
                 grid_steps: int = 100, convertible_engine_steps: int = 100):
        if not isinstance(bond_static, QuantLibBondStaticBase):
            raise TypeError("Requires QuantLibBondStaticBase derivative.")
        super().__init__(bond_static)
        self.method, self.grid_steps, self.convertible_engine_steps = method.lower(), grid_steps, convertible_engine_steps
        # Corrected attribute names:
        self.is_callable = isinstance(bond_static, CallableBondStaticBase)
        self.is_convertible = isinstance(bond_static, ConvertibleBondStaticBase)


    def _make_term_structure(self, pillar_times: np.ndarray, rates_vec: np.ndarray) -> ql.YieldTermStructureHandle:
        p_times_np = np.asarray(pillar_times, float)
        base_d: ql.Date = self.product_static.ql_valuation_date
        dates, eff_rates = ql.DateVector(), list(rates_vec)
        if not p_times_np.size or (p_times_np.size > 0 and p_times_np[0] > 1e-6):
             dates.push_back(base_d); eff_rates.insert(0, rates_vec[0] if rates_vec.size > 0 else 0.0)
        for t_val in p_times_np: dates.push_back(base_d + ql.Period(int(round(t_val*365.0)), ql.Days))
        if len(eff_rates) != len(dates):
             if len(dates)==1 and not eff_rates: eff_rates.append(0.0)

        cal = self.product_static.calendar_ql if hasattr(self.product_static, 'calendar_ql') else ql.TARGET()
        dc = self.product_static.day_count_ql if hasattr(self.product_static, 'day_count_ql') else ql.ActualActual(ql.ActualActual.ISDA)

        zc = ql.ZeroCurve(dates, eff_rates, dc, cal, ql.Linear(), ql.Continuous, ql.Annual)
        zc.enableExtrapolation(); return ql.YieldTermStructureHandle(zc)

    @staticmethod
    def _price_vanilla_static(bond: ql.Bond, ts_handle: ql.YieldTermStructureHandle) -> float:
        eng = ql.DiscountingBondEngine(ts_handle); bond.setPricingEngine(eng); return bond.NPV()
    @staticmethod
    def _price_callable_static(bond: ql.CallableFixedRateBond, ts_handle: ql.YieldTermStructureHandle,
                               params: tuple, steps: int) -> float:
        a,sig,b,eta,rho=params; model=ql.G2(ts_handle,a,sig,b,eta,rho)
        eng=ql.TreeCallableFixedRateBondEngine(model,steps); bond.setPricingEngine(eng); return bond.cleanPrice()
    @staticmethod
    def _price_convertible_static(
        bond: ql.ConvertibleFixedCouponBond, ts_handle: ql.YieldTermStructureHandle,
        static_def: ConvertibleBondStaticBase, eng_steps: int,
        s0: float, div_yield: float, eq_vol: float, credit_spread: float ) -> float:
        eval_d = ql.Settings.instance().evaluationDate
        s0_h = ql.QuoteHandle(ql.SimpleQuote(s0))
        day_count = static_def.day_count_ql
        calendar = static_def.calendar_ql
        div_h = ql.YieldTermStructureHandle(ql.FlatForward(eval_d, div_yield, day_count))
        vol_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_d, calendar, eq_vol, day_count))
        cs_h = ql.QuoteHandle(ql.SimpleQuote(credit_spread))
        proc = ql.BlackScholesMertonProcess(s0_h, div_h, ts_handle, vol_h)
        eng = ql.BinomialCRRConvertibleEngine(proc, eng_steps, cs_h); bond.setPricingEngine(eng)
        return bond.NPV()

    def price(self, pillar_times: np.ndarray, market_scenario_data: np.ndarray, **kwargs) -> np.ndarray:
        if market_scenario_data.ndim == 1: market_scenario_data = market_scenario_data.reshape(1, -1)
        prices = []
        is_g2_single = self.method=='g2' and (isinstance(kwargs.get('g2_params'), tuple) or (isinstance(kwargs.get('g2_params'), list) and kwargs.get('g2_params') and isinstance(kwargs.get('g2_params')[0], (float,int))))
        if self.method=='g2' and kwargs.get('g2_params') is not None and not is_g2_single and len(kwargs.get('g2_params')) != market_scenario_data.shape[0]:
            raise ValueError("List of g2_params must match number of scenarios for G2 method.")

        for i, scen_data_row in enumerate(market_scenario_data):
            current_g2_p = None
            if self.method == 'g2':
                 current_g2_p = kwargs.get('g2_params') if is_g2_single else (kwargs.get('g2_params')[i] if kwargs.get('g2_params') else None)
            prices.append(self._price_single_curve_logic(pillar_times, scen_data_row, current_g2_p, **kwargs))
        return np.array(prices)

    def _price_single_curve_logic(self,
                                  pillar_times_np: np.ndarray,
                                  market_data_for_scenario_row: np.ndarray,
                                  g2_p_for_this_scen=None, **other_fixed_params) -> float:
        ql.Settings.instance().evaluationDate = self.product_static.ql_valuation_date
        num_rate_pillars = len(pillar_times_np)
        rates_scen = market_data_for_scenario_row[:num_rate_pillars]
        ts_handle = self._make_term_structure(pillar_times_np, rates_scen)

        if self.is_convertible and self.method == 'convertible_binomial':
            if not isinstance(self.product_static, ConvertibleBondStaticBase):
                raise TypeError("Product is not ConvertibleBondStaticBase for CB pricing.")

            # Determine how many dynamic non-rate factors are present in market_data_for_scenario_row
            # This count helps determine if S0, Div, Vol, CS are coming from scenario or fixed_params
            num_dynamic_non_rate_factors = len(market_data_for_scenario_row) - num_rate_pillars

            # Initialize with fixed parameters from kwargs (passed from TFF training or full pricer call)
            s0_val = other_fixed_params.get('s0_val')
            div_val = other_fixed_params.get('dividend_yield')
            eq_vol_val = other_fixed_params.get('equity_volatility')
            cs_val = other_fixed_params.get('credit_spread')

            current_idx_in_scen_data = num_rate_pillars # Start reading after rate factors

            # If S0 is a dynamic TFF factor (always for CB TFFs in current TFFCalibrate setup)
            if num_dynamic_non_rate_factors >= 1:
                s0_val = market_data_for_scenario_row[current_idx_in_scen_data]
                current_idx_in_scen_data +=1

            # If other market inputs (Div, Vol, CS) are also dynamic TFF factors
            if num_dynamic_non_rate_factors >= 2:
                div_val = market_data_for_scenario_row[current_idx_in_scen_data]
                current_idx_in_scen_data +=1
            if num_dynamic_non_rate_factors >= 3:
                eq_vol_val = market_data_for_scenario_row[current_idx_in_scen_data]
                current_idx_in_scen_data +=1
            if num_dynamic_non_rate_factors >= 4:
                cs_val = market_data_for_scenario_row[current_idx_in_scen_data]
                # current_idx_in_scen_data +=1 # Not needed for the last one

            # Final check for required parameters
            if s0_val is None: raise ValueError("S0 value missing for convertible bond pricing.")
            if div_val is None: raise ValueError("Dividend yield missing for convertible bond pricing.")
            if eq_vol_val is None: raise ValueError("Equity volatility missing for convertible bond pricing.")
            if cs_val is None: raise ValueError("Credit spread missing for convertible bond pricing.")

            return self._price_convertible_static(
                self.product_static.bond, ts_handle, self.product_static,
                self.convertible_engine_steps, s0_val, div_val, eq_vol_val, cs_val)

        elif self.is_callable and self.method == 'g2':
            if g2_p_for_this_scen is None: raise ValueError("g2_params needed for G2 pricing.")
            # Ensure self.product_static.bond is ql.CallableFixedRateBond
            if not isinstance(self.product_static.bond, ql.CallableFixedRateBond):
                 raise TypeError(f"Expected ql.CallableFixedRateBond, got {type(self.product_static.bond)}")
            return self._price_callable_static(self.product_static.bond, ts_handle, g2_p_for_this_scen, self.grid_steps)
        elif self.method == 'discount':
            # Ensure self.product_static.bond is ql.Bond (or FixedRateBond)
            if not isinstance(self.product_static.bond, ql.Bond):
                 raise TypeError(f"Expected ql.Bond, got {type(self.product_static.bond)}")
            return self._price_vanilla_static(self.product_static.bond, ts_handle)
        raise ValueError(f"Unsupported method '{self.method}' or product for QL BondPricer.")


class BlackScholesPricer(PricerBase):
    def __init__(self, option_static: EuropeanOptionStatic,
                 risk_free_rate: float, dividend_yield: float = 0.0):
        if not isinstance(option_static, EuropeanOptionStatic):
            raise TypeError("Requires EuropeanOptionStatic.")
        super().__init__(option_static)
        self.risk_free_rate, self.dividend_yield = risk_free_rate, dividend_yield
    def price(self, stock_price: np.ndarray, volatility: np.ndarray, **kwargs) -> np.ndarray:
        S, sig = np.asarray(stock_price), np.asarray(volatility)
        opt: EuropeanOptionStatic = self.product_static
        K, T, r, q,otype = opt.strike_price,opt.time_to_expiry,self.risk_free_rate,self.dividend_yield,opt.option_type
        is_scalar = (S.ndim==0 and sig.ndim==0)
        if S.ndim==0 and sig.ndim>0: S=np.full_like(sig,S)
        if sig.ndim==0 and S.ndim>0: sig=np.full_like(S,sig)
        price = np.zeros_like(S,float) if not is_scalar else 0.0
        if T<=1e-9: price = np.maximum(S-K if otype=='call' else K-S,0.0); return price
        sig_calc = np.maximum(sig,1e-16)
        with np.errstate(divide='ignore',invalid='ignore'):
            d1=(np.log(S/K)+(r-q+0.5*sig_calc**2)*T)/(sig_calc*np.sqrt(T))
            d2=d1-sig_calc*np.sqrt(T)
            d1=np.where(S<=0,-np.inf if otype=='call' else np.inf,d1)
            d2=np.where(S<=0,-np.inf if otype=='call' else np.inf,d2)
            d1=np.nan_to_num(d1,nan=-np.inf if otype=='call' else np.inf,posinf=np.inf,neginf=-np.inf)
            d2=np.nan_to_num(d2,nan=-np.inf if otype=='call' else np.inf,posinf=np.inf,neginf=-np.inf)
        price_bs = (S*np.exp(-q*T)*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)) if otype=='call' \
              else (K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(-d1))
        price_bs = np.nan_to_num(price_bs,nan=0.0)
        if np.any(sig<=1e-9):
            intr = np.maximum(S*np.exp(-q*T)-K*np.exp(-r*T) if otype=='call' else K*np.exp(-r*T)-S*np.exp(-q*T),0.0)
            if is_scalar and (sig<=1e-9): price=intr
            elif not is_scalar: price=np.where(sig<=1e-9,intr,price_bs)
            else: price=np.where(sig<=1e-9,intr,price_bs)
        else: price=price_bs
        return np.maximum(price,0.0)


In [96]:

# g2_model.py
"""
Contains the G2Calibrator class for calibrating the G2++ interest rate model.
"""
import QuantLib as ql

class G2Calibrator:
    """
    Calibrates a G2++ (two-factor Hull-White) interest rate model
    to market cap/floor quotes.

    The G2++ model has 5 parameters: a, sigma, b, eta, rho.
    - a: mean reversion speed of the first factor (x)
    - sigma: volatility of the first factor (x)
    - b: mean reversion speed of the second factor (y)
    - eta: volatility of the second factor (y)
    - rho: correlation between the Wiener processes driving x and y

    Args:
        ts_handle (ql.YieldTermStructureHandle): Handle to the initial yield term structure
                                                 to which the model will be fitted. This term
                                                 structure defines the initial discount curve.
        index (ql.IborIndex or ql.OvernightIndex): The interest rate index underlying the
                                                   calibration instruments (e.g., caps/floors).
                                                   This index is used by the CapHelper to
                                                   determine caplet cashflows and fixings.
    """
    def __init__(self, ts_handle: ql.YieldTermStructureHandle, index: ql.InterestRateIndex):
        self.ts_handle: ql.YieldTermStructureHandle = ts_handle
        self.index: ql.InterestRateIndex = index
        # Initialize a G2 model. The parameters (a, sigma, b, eta, rho) will be calibrated.
        # The model is initialized with the provided term structure.
        self.model: ql.G2 = ql.G2(self.ts_handle)

    def calibrate(
        self,
        periods: list[ql.Period],
        quotes: list[ql.QuoteHandle],
        optimization_method: ql.OptimizationMethod = None,
        end_criteria: ql.EndCriteria = None,
        engine_steps: int = 50
    ) -> tuple[float, float, float, float, float]:
        """
        Calibrates the G2 model parameters to market cap volatilities.

        Args:
            periods (list[ql.Period]): List of ql.Period objects for cap tenors
                                       (e.g., [ql.Period('1Y'), ql.Period('2Y'), ...]).
                                       There should be at least 5 instruments for 5 parameters.
            quotes (list[ql.QuoteHandle]): List of ql.QuoteHandle objects containing the market
                                           volatilities (or prices) for the corresponding periods.
                                           Assumes Normal volatility for caps.
            optimization_method (ql.OptimizationMethod, optional):
                                QuantLib optimization method (e.g., ql.LevenbergMarquardt).
                                Defaults to a standard LevenbergMarquardt.
            end_criteria (ql.EndCriteria, optional):
                                QuantLib end criteria for the optimization.
                                Defaults to a standard EndCriteria.
            engine_steps (int, optional): Number of time steps for the TreeCapFloorEngine
                                          used to price caps during calibration. Affects accuracy
                                          and speed of calibration. Defaults to 50.

        Returns:
            tuple[float, float, float, float, float]:
                A tuple containing the calibrated G2 model parameters: (a, sigma, b, eta, rho).

        Raises:
            ValueError: If the number of periods and quotes do not match.
            RuntimeError: If calibration fails in QuantLib (e.g., due to insufficient instruments
                          or optimization issues).
        """
        if len(periods) != len(quotes):
            raise ValueError("Length of periods and quotes must match for calibration.")
        if len(periods) < 5: # G2++ has 5 parameters
            # This is a common cause of calibration failure.
            print(f"Warning: Number of calibration instruments ({len(periods)}) is less than 5. "
                  "G2++ calibration might be unstable or fail due to under-specification.")

        helpers = []
        for period_obj, quote_handle in zip(periods, quotes):
            # Create a CapHelper for each market quote.
            # The CapHelper links the market quote (volatility) to the model price of the cap.
            # Parameters like cap frequency, day count should ideally match the market convention
            # of the provided quotes.
            helper = ql.CapHelper(
                period_obj,
                quote_handle,
                self.index,                     # IborIndex underlying the cap
                ql.Semiannual,                 # Cap/floor coupon frequency (e.g., Semiannual)
                self.index.dayCounter() if self.index.dayCounter() else ql.Actual360(), # Day count for caplets
                False,                         # Not used for caps (related to first fixing)
                self.ts_handle,                # Initial term structure for discounting
                ql.BlackCalibrationHelper.RelativePriceError, # Error type for calibration objective function
                ql.Normal,                     # Volatility type (e.g., Normal, ShiftedLognormal)
                0.0                            # Shift (if using ShiftedLognormal)
            )
            # The pricing engine for the helper uses the G2 model instance that is being calibrated.
            # engine_steps for the tree engine affects accuracy/speed of pricing each cap during calibration.
            engine = ql.TreeCapFloorEngine(self.model, engine_steps)
            helper.setPricingEngine(engine)
            helpers.append(helper)

        # Default optimization method and end criteria if not provided
        opt_method = optimization_method or ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
        crit = end_criteria or ql.EndCriteria(
            maxIterations=10000,
            maxStationaryStateIterations=1000,
            rootEpsilon=1e-8,
            functionEpsilon=1e-8,
            gradientNormEpsilon=1e-8
        )

        # Perform the calibration. This modifies self.model in-place.
        self.model.calibrate(helpers, opt_method, crit)

        # Return the calibrated parameters from the model
        # params() returns [a, sigma, b, eta, rho]
        return self.model.params()


In [97]:
# scenario_generator.py
"""
Contains classes for generating market scenarios for different risk factors.
SimpleRandomScenarioGenerator can now generate scenarios for a targeted list of factors.
"""
import numpy as np
import abc

class ScenarioGeneratorBase(abc.ABC):
    """
    Abstract base class for scenario generators.
    """
    @abc.abstractmethod
    def generate_scenarios(self, num_scenarios: int, target_factor_names: list[str] = None) -> tuple[np.ndarray, list[str]]:
        """
        Generates market scenarios.

        Args:
            num_scenarios (int): The number of scenarios to generate.
            target_factor_names (list[str], optional): A specific list of factor names
                to generate scenarios for. If None, generates for all configured factors.

        Returns:
            tuple[np.ndarray, list[str]]:
                - A 2D NumPy array where rows are scenarios and columns are risk factors.
                - A list of strings representing the names of the risk factors (columns),
                  matching the order of target_factor_names if provided, otherwise self.factor_names_ordered.
        """
        pass

class SimpleRandomScenarioGenerator(ScenarioGeneratorBase):
    """
    Generates scenarios using simple random shocks around base values.
    Can generate for all configured factors or a targeted subset.
    """
    def __init__(self,
                 base_rates_map: dict[str, float] = None,
                 rate_factor_shock_std_dev_map: dict[str, float] = None,
                 base_s0_map: dict[str, float] = None,
                 s0_shock_config_map: dict[str, tuple[str, float]] = None,
                 base_vol_map: dict[str, float] = None,
                 vol_shock_config_map: dict[str, tuple[str, float]] = None,
                 default_rate_shock_std_dev: float = 0.0010,
                 default_s0_shock_config: tuple[str, float] = ('normal', 0.10),
                 default_vol_shock_config: tuple[str, float] = ('normal', 0.05),
                 random_seed: int = None):

        self.base_rates_map = base_rates_map if base_rates_map is not None else {}
        self.rate_factor_shock_std_dev_map = rate_factor_shock_std_dev_map if rate_factor_shock_std_dev_map is not None else {}
        self.base_s0_map = base_s0_map if base_s0_map is not None else {}
        self.s0_shock_config_map = s0_shock_config_map if s0_shock_config_map is not None else {}
        self.base_vol_map = base_vol_map if base_vol_map is not None else {}
        self.vol_shock_config_map = vol_shock_config_map if vol_shock_config_map is not None else {}

        self.default_rate_shock_std_dev = default_rate_shock_std_dev
        self.default_s0_shock_config = default_s0_shock_config
        self.default_vol_shock_config = default_vol_shock_config

        self.rng = np.random.default_rng(random_seed)

        # Stores all unique factor names this generator is configured for, in a defined order.
        self._configured_factor_names_ordered = sorted(list(
            set(self.base_rates_map.keys()) |
            set(self.base_s0_map.keys()) |
            set(self.base_vol_map.keys())
        ))
        if not self._configured_factor_names_ordered and (self.base_rates_map or self.base_s0_map or self.base_vol_map):
            raise ValueError("Could not determine ordered factor names. Ensure keys in maps are strings.")

    def generate_scenarios(self, num_scenarios: int, target_factor_names: list[str] = None) -> tuple[np.ndarray, list[str]]:
        """
        Generates scenarios. If target_factor_names is provided, only generates for those factors.
        Otherwise, generates for all configured factors.
        """
        if target_factor_names is None:
            factors_to_generate = self._configured_factor_names_ordered
        else:
            # Validate that all target_factor_names are known to this generator
            for name in target_factor_names:
                if not (name in self.base_rates_map or \
                        name in self.base_s0_map or \
                        name in self.base_vol_map):
                    raise ValueError(f"Target factor name '{name}' is not configured in this scenario generator. "
                                     f"Known factors: {self._configured_factor_names_ordered}")
            factors_to_generate = target_factor_names # Generate in the requested order

        if not factors_to_generate:
            return np.array([]).reshape(num_scenarios, 0), []

        all_scenario_columns = []

        for factor_name in factors_to_generate:
            factor_column = np.zeros(num_scenarios)

            if factor_name in self.base_rates_map:
                base_value = self.base_rates_map[factor_name]
                shock_std_dev = self.rate_factor_shock_std_dev_map.get(factor_name, self.default_rate_shock_std_dev)
                shocks = self.rng.normal(loc=0.0, scale=shock_std_dev, size=num_scenarios)
                factor_column = base_value + shocks

            elif factor_name in self.base_s0_map:
                base_value = self.base_s0_map[factor_name]
                shock_type, shock_param = self.s0_shock_config_map.get(factor_name, self.default_s0_shock_config)
                if shock_type.lower() == 'normal':
                    actual_std_dev = shock_param * base_value if shock_param <= 1.0 else shock_param
                    shocks = self.rng.normal(loc=0.0, scale=actual_std_dev, size=num_scenarios)
                    factor_column = base_value + shocks
                elif shock_type.lower() == 'uniform':
                    half_width = shock_param * base_value if shock_param <= 1.0 else shock_param
                    factor_column = self.rng.uniform(base_value - half_width, base_value + half_width, size=num_scenarios)
                else:
                    raise ValueError(f"Unsupported shock_type: {shock_type} for S0 factor {factor_name}")
                factor_column = np.maximum(factor_column, 1e-6)

            elif factor_name in self.base_vol_map:
                base_value = self.base_vol_map[factor_name]
                shock_type, shock_param = self.vol_shock_config_map.get(factor_name, self.default_vol_shock_config)
                if shock_type.lower() == 'normal':
                    actual_std_dev = shock_param * base_value if shock_param <= 1.0 else shock_param
                    shocks = self.rng.normal(loc=0.0, scale=actual_std_dev, size=num_scenarios)
                    factor_column = base_value + shocks
                elif shock_type.lower() == 'uniform':
                    half_width = shock_param * base_value if shock_param <= 1.0 else shock_param
                    factor_column = self.rng.uniform(base_value - half_width, base_value + half_width, size=num_scenarios)
                else:
                    raise ValueError(f"Unsupported shock_type: {shock_type} for Vol factor {factor_name}")
                factor_column = np.maximum(factor_column, 1e-6)

            else:
                # This case should ideally not be reached if target_factor_names are validated
                # or if using _configured_factor_names_ordered.
                raise ValueError(f"Factor name {factor_name} not found in any configuration map during generation.")

            all_scenario_columns.append(factor_column[:, np.newaxis])

        return np.hstack(all_scenario_columns), factors_to_generate # Return the names of factors generated


In [105]:
# tff_approximator.py
"""
Contains classes and functions for Tensor Functional Form (TFF) approximation.
Convertible TFF factor handling refined.
Date handling in worker function uses isoformat.
Corrected TensorFunctionalFormCalibrate init keyword.
"""
import numpy as np
from scipy.stats.qmc import LatinHypercube, Sobol, scale
from concurrent.futures import ProcessPoolExecutor
import QuantLib as ql
from datetime import date, datetime
import re
import os

# --- Feature Engineering and Normalization for Options ---
def engineer_option_features(
    s0_values: np.ndarray, vol_values: np.ndarray, order: int = 2
) -> tuple[np.ndarray, list[str]]:
    s0, vol = np.asarray(s0_values), np.asarray(vol_values)
    if s0.shape != vol.shape or s0.ndim != 1: raise ValueError("Invalid s0/vol shapes.")
    features = [s0, vol]; feature_names = ['S0_eng', 'Vol_eng']
    if order >= 2:
        features.extend([s0**2, vol**2, s0 * vol])
        feature_names.extend(['S0^2_eng', 'Vol^2_eng', 'S0*Vol_eng'])
    if order >= 3:
        features.extend([s0**3, vol**3, (s0**2)*vol, s0*(vol**2)])
        feature_names.extend(['S0^3_eng', 'Vol^3_eng', 'S0^2*Vol_eng', 'S0*Vol^2_eng'])
    if order >= 4:
        features.extend([s0**4, vol**4, (s0**3)*vol, s0*(vol**3), (s0**2)*(vol**2)])
        feature_names.extend(['S0^4_eng', 'Vol^4_eng', 'S0^3*Vol_eng', 'S0*Vol^3_eng', 'S0^2*Vol^2_eng'])
    return np.vstack(features).T, feature_names

def normalize_features(
    features: np.ndarray, means: np.ndarray = None, stds: np.ndarray = None
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    if features.ndim != 2: raise ValueError("Features must be 2D array.")
    if means is None or stds is None:
        means, stds = np.mean(features, axis=0), np.std(features, axis=0)
        stds[stds < 1e-8] = 1.0
    return (features - means) / stds, means, stds

def _parse_numeric_pillars_from_factor_names(factor_names: list[str]) -> np.ndarray:
    parsed_pillars = []
    for name_str in factor_names:
        match = re.search(r'(\d+(\.\d+)?)(?=Y)', name_str)
        if not match: match = re.search(r'(\d+(\.\d+)?)', name_str)
        if match:
            try:
                if any(sub.upper() in name_str.upper() for sub in ["RATE", "IR", "CURVE", "YIELD"]) and "Y" in name_str.upper():
                    parsed_pillars.append(float(match.group(1)))
            except ValueError: pass
        else:
            try:
                if not any(equity_tag in name_str.upper() for equity_tag in ["S0", "VOL", "EQUITY", "STOCK", "DIVYIELD", "CS"]):
                    parsed_pillars.append(float(name_str))
            except ValueError: pass
    return np.array(sorted(list(set(parsed_pillars))), dtype=float) if parsed_pillars else np.array([], dtype=float)


class TensorFunctionalForm:
    def __init__(self, A: np.ndarray, b: np.ndarray, c: float):
        self.A, self.b, self.c = np.asarray(A,float), np.asarray(b,float), float(c)
        if self.A.ndim!=2 or self.A.shape[0]!=self.A.shape[1]: raise ValueError("A must be square.")
        if self.b.ndim!=1 or self.b.shape[0]!=self.A.shape[0]: raise ValueError("b dim must match A.")
        self.D: int = self.A.shape[0]
    def __call__(self, x: np.ndarray) -> np.ndarray:
        x_arr = np.asarray(x)
        if x_arr.ndim == 1:
            if x_arr.shape[0]!=self.D: raise ValueError(f"Input dim {x_arr.shape[0]} != model dim {self.D}.")
            return float(x_arr @ self.A @ x_arr + self.b @ x_arr + self.c)
        elif x_arr.ndim == 2:
            if x_arr.shape[1]!=self.D: raise ValueError(f"Input shape {x_arr.shape}, expected (N, {self.D}).")
            return np.sum((x_arr @ self.A) * x_arr, axis=1) + x_arr @ self.b + self.c
        raise ValueError(f"Input must be 1D or 2D, got ndim={x_arr.ndim}")
    def to_dict(self) -> dict: return {'A':self.A.tolist(),'b':self.b.tolist(),'c':self.c,'D':self.D}
    @classmethod
    def from_dict(cls, data:dict) -> 'TensorFunctionalForm':
        if not all(k in data for k in ['A','b','c']): raise ValueError("Missing keys in TFF data dict.")
        return cls(np.array(data['A'],float), np.array(data['b'],float), data['c'])


def _price_one_scenario_for_tff(worker_args: tuple) -> float:
    (product_static_params_dict, pricer_config_params,
     factor_names_for_tff, single_market_scenario_data,
     valuation_date_for_worker_iso, price_kwargs_dict) = worker_args

    valuation_date_for_worker = _parse_date_input(valuation_date_for_worker_iso)
    ql_val_date = ql.Date(valuation_date_for_worker.day, valuation_date_for_worker.month, valuation_date_for_worker.year)
    ql.Settings.instance().evaluationDate = ql_val_date

    product_type = product_static_params_dict['product_type']
    actual_rate_pillars_for_worker = np.asarray(product_static_params_dict.get('actual_rate_pillars', []), dtype=float)

    current_static_params = product_static_params_dict.copy()
    # valuation_date in current_static_params is already an ISO string from product_static_object.to_dict()
    # The from_dict methods in product_definitions will parse it.

    if product_type == 'VanillaBond': product_static_obj = QuantLibBondStaticBase.from_dict(current_static_params)
    elif product_type == 'CallableBond': product_static_obj = CallableBondStaticBase.from_dict(current_static_params)
    elif product_type == 'ConvertibleBond': product_static_obj = ConvertibleBondStaticBase.from_dict(current_static_params)
    elif product_type == 'EuropeanOption': product_static_obj = EuropeanOptionStatic.from_dict(current_static_params)
    else: raise ValueError(f"Unknown product type for TFF worker: {product_type}")

    if product_type in ['VanillaBond', 'CallableBond', 'ConvertibleBond']:
        pricer_instance = QuantLibBondPricer(product_static_obj, **pricer_config_params.get('bond_pricer_config',{}))
        market_data_for_ql_pricer = np.array([single_market_scenario_data])

        price_result_array = pricer_instance.price(
            pillar_times=actual_rate_pillars_for_worker,
            market_scenario_data=market_data_for_ql_pricer,
            **price_kwargs_dict
        )
        return price_result_array[0]
    elif product_type == 'EuropeanOption':
        bs_config = pricer_config_params.get('bs_pricer_config', {})
        pricer_instance = BlackScholesPricer(product_static_obj, bs_config['risk_free_rate'], bs_config.get('dividend_yield',0.0))
        if len(single_market_scenario_data) == 2: # S0 and Vol
            return pricer_instance.price(stock_price=single_market_scenario_data[0], volatility=single_market_scenario_data[1])
        else: raise ValueError(f"Option TFF expects 2 inputs (S0, Vol), got {len(single_market_scenario_data)}")
    raise ValueError(f"Pricer path failed for product type: {product_type}")


class TensorFunctionalFormCalibrate:
    def __init__(
        self, pricer_template: PricerBase, scenario_generator: SimpleRandomScenarioGenerator,
        bond_numeric_tenors: np.ndarray = None,
        # Corrected parameter name
        convertible_tff_market_inputs_as_factors: bool = False
    ):
        self.pricer_template = pricer_template
        self.product_static: ProductStaticBase = pricer_template.product_static
        self.scenario_generator = scenario_generator
        self.product_type_str: str = None

        self.tff_input_raw_factor_names: list[str] = []
        self.tff_input_raw_base_values: np.ndarray = np.array([])
        self.valuation_date_for_ql_settings_in_worker: date = self.product_static.valuation_date_py
        self.product_static_params_for_worker: dict = {}
        self.pricer_config_for_worker: dict = {}
        self.actual_rate_pillars: np.ndarray = None
        self.convertible_tff_market_inputs_as_factors = convertible_tff_market_inputs_as_factors


        if isinstance(self.product_static, EuropeanOptionStatic):
            self.product_type_str = 'EuropeanOption'
            if not self.product_static.underlying_symbol or not self.product_static.currency:
                raise ValueError("EuropeanOptionStatic needs 'underlying_symbol' and 'currency'.")
            s0_fn = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_S0"
            vol_fn = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_VOL"
            self.tff_input_raw_factor_names = [s0_fn, vol_fn]
            if s0_fn not in scenario_generator.base_s0_map or vol_fn not in scenario_generator.base_vol_map:
                raise ValueError(f"S0/Vol factors '{s0_fn}', '{vol_fn}' not in scenario_generator maps. Available S0: {list(scenario_generator.base_s0_map.keys())}, Available Vol: {list(scenario_generator.base_vol_map.keys())}")
            self.tff_input_raw_base_values = np.array([scenario_generator.base_s0_map[s0_fn], scenario_generator.base_vol_map[vol_fn]])
            self.actual_rate_pillars = np.array([])
            self.product_static_params_for_worker = self.product_static.to_dict()
            if isinstance(self.pricer_template, BlackScholesPricer):
                 self.pricer_config_for_worker['bs_pricer_config'] = {
                    'risk_free_rate': self.pricer_template.risk_free_rate, 'dividend_yield': self.pricer_template.dividend_yield}
            else: raise TypeError("Pricer for EuropeanOptionStatic must be BlackScholesPricer.")

        elif isinstance(self.product_static, QuantLibBondStaticBase):
            if not self.product_static.currency: raise ValueError("Bond product needs 'currency'.")
            if not self.product_static.index_stub: raise ValueError(f"Bond product '{self.product_static.__class__.__name__}' needs a non-empty 'index_stub'. Got: '{self.product_static.index_stub}'")
            if bond_numeric_tenors is None or bond_numeric_tenors.size == 0:
                raise ValueError("bond_numeric_tenors needed for Bond TFF.")

            rate_fn = [f"{self.product_static.currency}_{self.product_static.index_stub}_{t:.2f}Y" for t in bond_numeric_tenors]
            base_rates = []
            for name in rate_fn:
                if name not in scenario_generator.base_rates_map: raise ValueError(f"Rate factor '{name}' not in scenario_generator base_rates_map. Available: {list(scenario_generator.base_rates_map.keys())}")
                base_rates.append(scenario_generator.base_rates_map[name])
            self.tff_input_raw_factor_names.extend(rate_fn)
            self.tff_input_raw_base_values = np.array(base_rates)
            self.actual_rate_pillars = _parse_numeric_pillars_from_factor_names(rate_fn)

            self.product_static_params_for_worker = self.product_static.to_dict()
            self.product_static_params_for_worker['actual_rate_pillars'] = self.actual_rate_pillars.tolist()

            if isinstance(self.pricer_template, QuantLibBondPricer):
                 self.pricer_config_for_worker['bond_pricer_config'] = {
                    'method': self.pricer_template.method, 'grid_steps': self.pricer_template.grid_steps,
                    'convertible_engine_steps': self.pricer_template.convertible_engine_steps}
            else: raise TypeError("Pricer for QuantLibBondStaticBase must be QuantLibBondPricer.")

            if isinstance(self.product_static, CallableBondStaticBase): self.product_type_str = 'CallableBond'
            elif isinstance(self.product_static, ConvertibleBondStaticBase):
                self.product_type_str = 'ConvertibleBond'
                if not self.product_static.underlying_symbol: raise ValueError("Convertible needs 'underlying_symbol'.")

                s0_factor_name_conv = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_S0"
                if s0_factor_name_conv not in self.scenario_generator.base_s0_map:
                    raise ValueError(f"S0 factor '{s0_factor_name_conv}' for CB not found in scenario_generator.base_s0_map.")
                if s0_factor_name_conv not in self.tff_input_raw_factor_names:
                    self.tff_input_raw_factor_names.append(s0_factor_name_conv)
                    self.tff_input_raw_base_values = np.append(self.tff_input_raw_base_values, self.scenario_generator.base_s0_map[s0_factor_name_conv])

                if self.convertible_tff_market_inputs_as_factors:
                    div_fn = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_DIVYIELD"
                    vol_fn = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_EQVOL"
                    cs_fn = f"{self.product_static.currency}_{self.product_static.underlying_symbol}_CS"

                    cb_market_factor_names = [div_fn, vol_fn, cs_fn]
                    cb_market_base_values = []

                    def get_base_val_from_sg(factor_name, sg):
                        maps_to_check = [sg.base_s0_map, sg.base_vol_map, sg.base_rates_map]
                        for m_map in maps_to_check:
                            if factor_name in m_map: return m_map[factor_name]
                        # Try a generic 'other_factors' map if it exists in scenario_generator
                        if hasattr(sg, 'base_other_factors_map') and factor_name in sg.base_other_factors_map:
                            return sg.base_other_factors_map[factor_name]
                        raise ValueError(f"Base value for TFF factor '{factor_name}' not found in scenario_generator configured maps.")

                    for fn in cb_market_factor_names:
                        cb_market_base_values.append(get_base_val_from_sg(fn, scenario_generator))

                    self.tff_input_raw_factor_names.extend(cb_market_factor_names)
                    self.tff_input_raw_base_values = np.append(self.tff_input_raw_base_values, cb_market_base_values)
            else: self.product_type_str = 'VanillaBond'

            self.product_static_params_for_worker['product_type'] = self.product_type_str

        else: raise TypeError(f"Unsupported pricer_template product type: {type(self.product_static)}")

        if not self.tff_input_raw_factor_names or self.tff_input_raw_base_values.size == 0:
            raise RuntimeError(f"TFF input factors/base values not set for {self.product_type_str}")
        if len(self.tff_input_raw_factor_names) != len(self.tff_input_raw_base_values):
            raise RuntimeError(f"Mismatch TFF factor names ({len(self.tff_input_raw_factor_names)}) and base values ({len(self.tff_input_raw_base_values)}). Names: {self.tff_input_raw_factor_names}, Base Values: {self.tff_input_raw_base_values}")

        for key, value in self.product_static_params_for_worker.items():
            if key in ['valuation_date', 'maturity_date', 'issue_date', 'expiry_date']:
                if isinstance(value, date) and not isinstance(value, ql.Date):
                    self.product_static_params_for_worker[key] = value.isoformat()
            elif key == 'call_dates' and isinstance(value, list):
                 self.product_static_params_for_worker[key] = [d.isoformat() if isinstance(d, date) else d for d in value]
            elif isinstance(value, np.ndarray):
                 self.product_static_params_for_worker[key] = value.tolist()


    def sample_and_fit(
        self, full_market_scenarios_for_tff_factors: np.ndarray,
        n_train: int = 50, n_test: int = 20,
        random_seed: int = 0, sampling_method: str = 'sobol', parallel_workers: int = None,
        option_feature_order: int = 0, **price_kwargs
    ) -> tuple[TensorFunctionalForm, np.ndarray, np.ndarray, float, dict]:

        rng_np = np.random.default_rng(random_seed)
        num_tff_factors = len(self.tff_input_raw_factor_names)

        if full_market_scenarios_for_tff_factors.ndim != 2 or \
           full_market_scenarios_for_tff_factors.shape[1] != num_tff_factors:
            raise ValueError(f"Shape error for scenarios. Expected (N, {num_tff_factors}), got {full_market_scenarios_for_tff_factors.shape}. Factors: {self.tff_input_raw_factor_names}")

        domain_min, domain_max = np.min(full_market_scenarios_for_tff_factors, axis=0), np.max(full_market_scenarios_for_tff_factors, axis=0)

        train_tff_inputs_raw = None
        if sampling_method == 'sobol':
            sampler = Sobol(d=num_tff_factors, scramble=True, seed=random_seed)
            train_tff_inputs_raw = scale(sampler.random(n=n_train), domain_min, domain_max)
        elif sampling_method == 'lhs':
            sampler_lhs = LatinHypercube(d=num_tff_factors, centered=True, seed=random_seed)
            train_tff_inputs_raw = scale(sampler_lhs.random(n=n_train), domain_min, domain_max)
        elif sampling_method == 'uniform':
            train_tff_inputs_raw = rng_np.uniform(low=domain_min, high=domain_max, size=(n_train, num_tff_factors))
        else: raise ValueError(f"Unknown sampling: {sampling_method}.")

        worker_args_list = [(self.product_static_params_for_worker, self.pricer_config_for_worker,
             self.tff_input_raw_factor_names, train_tff_inputs_raw[i],
             self.valuation_date_for_ql_settings_in_worker.isoformat(),
             price_kwargs) for i in range(n_train)]

        if parallel_workers is not False and parallel_workers != 0 and n_train > 1:
            actual_workers = parallel_workers if parallel_workers else (os.cpu_count() if os.cpu_count() else 1)
            print(f"   Generating {n_train} training prices in parallel (workers={actual_workers})...")
            with ProcessPoolExecutor(max_workers=actual_workers) as executor:
                train_prices = np.array(list(executor.map(_price_one_scenario_for_tff, worker_args_list)))
        else:
            print(f"   Generating {n_train} training prices sequentially...")
            train_prices = np.array([_price_one_scenario_for_tff(args) for args in worker_args_list])

        if train_prices.ndim == 0 and n_train == 1: train_prices = np.array([train_prices])
        if train_prices.shape[0] != n_train: raise ValueError(f"Shape of train_prices {train_prices.shape} != n_train {n_train}")

        tff_inputs_for_fitting = train_tff_inputs_raw
        normalization_params = {'means': None, 'stds': None, 'engineered_feature_names': self.tff_input_raw_factor_names, 'is_engineered': False}

        if self.product_type_str == 'EuropeanOption' and option_feature_order > 0:
            print(f"   Engineering features for option TFF (order={option_feature_order})...")
            if train_tff_inputs_raw.shape[1] != 2: raise ValueError(f"Option FE expects 2 raw inputs, got {train_tff_inputs_raw.shape[1]}")
            engineered_features_train, eng_names = engineer_option_features(train_tff_inputs_raw[:,0], train_tff_inputs_raw[:,1], order=option_feature_order)
            tff_inputs_for_fitting, means, stds = normalize_features(engineered_features_train)
            normalization_params = {'means':means.tolist(), 'stds':stds.tolist(), 'engineered_feature_names':eng_names, 'is_engineered':True}

        D_eff = tff_inputs_for_fitting.shape[1]
        X_train = np.hstack([np.array([np.outer(s,s).flatten() for s in tff_inputs_for_fitting]), tff_inputs_for_fitting, np.ones((n_train,1))])
        if np.any(np.isnan(X_train)) or np.any(np.isinf(X_train)): raise ValueError("NaN/Inf in X_train.")
        if np.any(np.isnan(train_prices)) or np.any(np.isinf(train_prices)): raise ValueError("NaN/Inf in train_prices.")
        try: coeffs,_,_,_ = np.linalg.lstsq(X_train, train_prices, rcond=None)
        except np.linalg.LinAlgError as e: raise np.linalg.LinAlgError(f"Lstsq failed: {e}.")

        A_flat = coeffs[:D_eff*D_eff]
        A_mat = A_flat.reshape(D_eff,D_eff); A_sym = 0.5*(A_mat+A_mat.T)
        b_vec, c_s = coeffs[D_eff*D_eff : D_eff*D_eff+D_eff], coeffs[-1]
        fitted_tff = TensorFunctionalForm(A_sym, b_vec, c_s)

        test_idx = rng_np.choice(full_market_scenarios_for_tff_factors.shape[0], size=n_test, replace=False)
        test_tff_inputs_raw = full_market_scenarios_for_tff_factors[test_idx]
        test_worker_args = [(self.product_static_params_for_worker, self.pricer_config_for_worker,
             self.tff_input_raw_factor_names, test_tff_inputs_raw[i],
             self.valuation_date_for_ql_settings_in_worker.isoformat(), price_kwargs) for i in range(n_test)]
        print(f"   Generating {n_test} test prices sequentially...")
        test_true_prices = np.array([_price_one_scenario_for_tff(args) for args in test_worker_args])

        test_inputs_eval = test_tff_inputs_raw
        if self.product_type_str == 'EuropeanOption' and normalization_params.get('is_engineered', False):
            if test_tff_inputs_raw.shape[1]!=2: raise ValueError(f"Test option TFF inputs expect 2 cols, got {test_tff_inputs_raw.shape[1]}")
            eng_feat_test,_ = engineer_option_features(test_tff_inputs_raw[:,0], test_tff_inputs_raw[:,1], order=option_feature_order)
            np_means = np.array(normalization_params['means']) if normalization_params.get('means') is not None else None
            np_stds = np.array(normalization_params['stds']) if normalization_params.get('stds') is not None else None
            test_inputs_eval,_,_ = normalize_features(eng_feat_test, np_means, np_stds)

        test_pred_prices = fitted_tff(test_inputs_eval)
        if test_true_prices.ndim==0 and n_test==1: test_true_prices = np.array([test_true_prices])
        if test_pred_prices.ndim==0 and n_test==1: test_pred_prices = np.array([test_pred_prices])
        if test_true_prices.shape != test_pred_prices.shape: raise ValueError(f"Shape mismatch test prices: true {test_true_prices.shape}, pred {test_pred_prices.shape}")

        rmse = np.sqrt(np.mean((test_true_prices - test_pred_prices)**2))
        return fitted_tff, test_tff_inputs_raw, test_true_prices, rmse, normalization_params


In [106]:
# utils.py
"""
Contains utility functions for the FastRiskDemo, such as generating
collections of bond definitions for portfolio testing.
"""
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

def generate_bond_collections(
    num_bonds: int,
    valuation_date: date = date(2025, 1, 1),
    face_value: float = 100.0,
    seed: int = 0,
    conv_params: dict = None # Parameters for generating convertibles
) -> tuple[list[QuantLibBondStaticBase], list[CallableBondStaticBase], list[ConvertibleBondStaticBase]]:
    """
    Generates collections of random vanilla, callable, and convertible bond static definitions.

    Args:
        num_bonds (int): Number of bond instances of each type to generate.
        valuation_date (date, optional): Common valuation date for all bonds.
                                         Defaults to date(2025, 1, 1).
        face_value (float, optional): Default face value for bonds. Defaults to 100.0.
        seed (int, optional): Random seed for reproducibility. Defaults to 0.
        conv_params (dict, optional): Dictionary of parameters for generating convertible bonds.
                                      Keys can include 'conversion_ratio', 'dividend_yield',
                                      'equity_volatility', 'initial_stock_price',
                                      'credit_spread_value', 'exercise_type'.
                                      Defaults are used if not provided.

    Returns:
        tuple[list, list, list]:
            A tuple containing:
            - A list of QuantLibBondStaticBase instances (vanilla bonds).
            - A list of CallableBondStaticBase instances (callable bonds).
            - A list of ConvertibleBondStaticBase instances (convertible bonds).
    """
    rng = np.random.default_rng(seed)
    vanilla_bonds = []
    callable_bonds = []
    convertible_bonds = []

    # Default parameters for convertible bond generation
    default_conv_params = {
        'conversion_ratio': 20.0,        # Example: 20 shares per bond of 100 face value
        'dividend_yield': 0.01,         # 1% continuous dividend yield
        'equity_volatility': 0.25,      # 25% annual volatility
        'initial_stock_price': 100.0,   # Base stock price
        'credit_spread_value': 0.015,   # 150 bps credit spread
        'exercise_type': 'EuropeanAtMaturity'
    }
    # Update defaults with any user-provided conv_params
    current_conv_params = default_conv_params.copy()
    if conv_params is not None:
        current_conv_params.update(conv_params)

    for i in range(num_bonds):
        # Common random parameters for each bond in the iteration
        years_to_maturity = int(rng.integers(3, 11)) # Ensure bonds have some life, min 3 years
        maturity_d = valuation_date + relativedelta(years=years_to_maturity)

        # Use different coupon ranges for different bond types for more realism
        van_call_coupon = float(rng.uniform(0.02, 0.06)) # 2% to 6% for vanilla/callable
        conv_coupon = float(rng.uniform(0.01, 0.04)) # 1% to 4% for convertibles (often lower)

        coupon_freq = int(rng.choice([1, 2]))    # Annual or Semi-annual

        # --- Vanilla Bond ---
        # Parameters for QuantLibBondStaticBase.from_dict
        vanilla_params_dict = {
            'valuation_date': valuation_date,
            'maturity_date': maturity_d,
            'coupon_rate': van_call_coupon,
            'face_value': face_value,
            'freq': coupon_freq,
            'settlement_days': 0
        }
        vanilla_bonds.append(QuantLibBondStaticBase.from_dict(vanilla_params_dict))

        # --- Callable Bond ---
        call_dates_list_py = []
        call_prices_list_py = []
        if years_to_maturity > 2: # Ensure some non-call period for callable bonds
            # Generate 1 to 3 call dates for variety
            num_calls = int(rng.integers(1, min(4, years_to_maturity - 1)))
            possible_call_years_offsets = list(range(1, years_to_maturity)) # Call can be from year 1 to year T-1

            if num_calls > 0 and len(possible_call_years_offsets) >= num_calls:
                chosen_call_years_offsets = sorted(rng.choice(possible_call_years_offsets, size=num_calls, replace=False))
                for year_offset in chosen_call_years_offsets:
                    call_d = valuation_date + relativedelta(years=int(year_offset))
                    if call_d < maturity_d: # Ensure call date is before maturity
                        call_dates_list_py.append(call_d)
                        # Call price typically at par or a slight premium
                        call_prices_list_py.append(float(face_value + rng.uniform(0.0, 3.0)))

        if call_dates_list_py and call_prices_list_py: # Only if valid call dates were generated
            callable_params_dict = {
                'valuation_date': valuation_date, 'maturity_date': maturity_d,
                'coupon_rate': van_call_coupon, 'face_value': face_value,
                'freq': coupon_freq, 'settlement_days': 0,
                'call_dates': call_dates_list_py, 'call_prices': call_prices_list_py
            }
            callable_bonds.append(CallableBondStaticBase.from_dict(callable_params_dict))

        # --- Convertible Bond ---
        # Issue date for convertible, can be different from valuation date.
        issue_offset_years = int(rng.integers(0, min(3, years_to_maturity - 1)))
        issue_date_convertible = valuation_date - relativedelta(years=issue_offset_years)
        # Ensure issue date is not after valuation date for this setup
        if issue_date_convertible > valuation_date:
            issue_date_convertible = valuation_date

        # Randomize some convertible parameters slightly for variety around the provided base
        s0_rand = current_conv_params['initial_stock_price'] * rng.uniform(0.8, 1.2)
        vol_rand = current_conv_params['equity_volatility'] * rng.uniform(0.7, 1.3)
        div_rand = max(0, current_conv_params['dividend_yield'] * rng.uniform(0.5, 1.5))
        cs_rand = max(0.001, current_conv_params['credit_spread_value'] * rng.uniform(0.5, 2.0))
        cr_rand = current_conv_params['conversion_ratio'] * rng.uniform(0.9, 1.1)

        convertible_params_dict = {
            'valuation_date': valuation_date,
            'issue_date': issue_date_convertible,
            'maturity_date': maturity_d,
            'coupon_rate': conv_coupon,
            'conversion_ratio': cr_rand,
            'dividend_yield': div_rand,
            'equity_volatility': vol_rand,
            'initial_stock_price': s0_rand,
            'credit_spread_value': cs_rand,
            'face_value': face_value,
            'freq': coupon_freq,
            'settlement_days': 0,
            'exercise_type': current_conv_params['exercise_type']
        }
        convertible_bonds.append(ConvertibleBondStaticBase.from_dict(convertible_params_dict))

    return vanilla_bonds, callable_bonds, convertible_bonds


In [107]:
# portfolio.py
"""
Contains classes for defining and analyzing portfolios of financial instruments.
The Portfolio class allows pricing using either TFF models (retrieved from a cache)
or full pricers.
"""
import numpy as np
import abc

class PortfolioBase(abc.ABC):
    """
    Abstract base class for a portfolio of financial instruments.
    """
    def __init__(self):
        # Stores details for each *position* in the portfolio.
        # Multiple positions can refer to the same underlying instrument_id if TFFs are cached.
        self.positions: list[dict] = []

    @abc.abstractmethod
    def add_position(self, *args, **kwargs):
        """Adds a position (instrument holding) to the portfolio."""
        pass

    @abc.abstractmethod
    def price_portfolio(self,
                        raw_market_scenarios: np.ndarray,
                        scenario_factor_names: list[str],
                        portfolio_rate_pillar_times: np.ndarray = None # For bond pricers
                        ) -> np.ndarray:
        """
        Prices all instruments in the portfolio for given market scenarios.

        Args:
            raw_market_scenarios (np.ndarray): 2D array of market scenarios
                                               (N_scenarios, N_total_market_factors).
            scenario_factor_names (list[str]): Names of the columns in raw_market_scenarios.
            portfolio_rate_pillar_times (np.ndarray, optional): 1D array of rate pillar times.
                                                               Required if portfolio contains bonds
                                                               priced with full QuantLibBondPricer.
        Returns:
            np.ndarray: 1D array of aggregated portfolio prices for each scenario (N_scenarios,).
        """
        pass

class Portfolio(PortfolioBase):
    """
    A portfolio where each instrument can be priced using either a pre-fitted
    Tensor Functional Form (TFF) model (retrieved from a cache) or its original full pricer.
    """
    def __init__(self):
        super().__init__()
        self.tff_model_cache: dict = {}
        # Cache structure:
        # { instrument_id: {
        #     'tff_model': TensorFunctionalForm_object,
        #     'raw_tff_input_names': list_of_names, # Raw factors TFF was trained on
        #     'normalization_params': dict_of_norm_params, # Includes engineered_feature_names
        #     'option_feature_order': int
        #   }, ...
        # }

    def to_dict(self) -> dict:
        """
        Returns a dictionary representation of the portfolio.

        Returns:
            dict: Dictionary representation of the portfolio.
        """
        return {
            'positions': [p.to_dict() if hasattr(p, 'to_dict') else p
                for p in self.positions],
            'tff_model_cache': self.tff_model_cache
        }

    def cache_tff_model(self,
                        instrument_id: str,
                        tff_model: TensorFunctionalForm,
                        raw_tff_input_names: list[str],
                        normalization_params: dict,
                        option_feature_order: int = 0):
        """
        Explicitly caches a fitted TFF model and its associated parameters.

        Args:
            instrument_id (str): Unique ID for the instrument type this TFF represents.
            tff_model (TensorFunctionalForm): The pre-fitted TFF model.
            raw_tff_input_names (list[str]): Names of raw market factors TFF is based on.
            normalization_params (dict): Normalization params from TFF calibration.
                                         Expected keys: 'means', 'stds', 'engineered_feature_names', 'is_engineered'.
            option_feature_order (int, optional): Order of feature engineering if option TFF.
        """
        if not instrument_id:
            raise ValueError("instrument_id must be provided for caching TFF model.")
        if not isinstance(tff_model, TensorFunctionalForm):
            raise TypeError("tff_model must be an instance of TensorFunctionalForm.")
        if raw_tff_input_names is None or not isinstance(raw_tff_input_names, list):
            raise ValueError("raw_tff_input_names (list of strings) must be provided for caching TFF model.")
        if normalization_params is None or not isinstance(normalization_params, dict):
            raise ValueError("normalization_params (dict) must be provided for caching TFF model.")

        self.tff_model_cache[instrument_id] = {
            'tff_model': tff_model,
            'raw_tff_input_names': raw_tff_input_names,
            'normalization_params': normalization_params,
            'option_feature_order': option_feature_order
        }

    def from_dict(self, portfolio_dict: dict):
        """
        Loads a portfolio from a dictionary representation. Similar to cache_tff_model

        Args:
            portfolio_dict (dict): Dictionary representation of the portfolio.
        """



    def add_position(self,
                       instrument_id: str,
                       product_static: ProductStaticBase,
                       num_holdings: int = 1,
                       pricing_engine_type: str = 'tff',
                       direct_tff_config: dict = None,
                       full_pricer_instance: PricerBase = None,
                       full_pricer_kwargs: dict = None):
        """
        Adds a position (an instrument holding) to the portfolio.
        If pricing_engine_type is 'tff', it retrieves the TFF model from the cache
        using the instrument_id. The TFF model must have been cached previously using `cache_tff_model`.

        Args:
            instrument_id (str): Unique ID for this instrument type. Used for TFF caching/lookup.
            product_static (ProductStaticBase): Static definition of the product for this position.
            num_holdings (int, optional): Number of units. Defaults to 1.
            pricing_engine_type (str, optional): 'tff' or 'full'. Defaults to 'tff'.
            full_pricer_instance (PricerBase, optional): A full pricer instance (required if type is 'full').
            full_pricer_kwargs (dict, optional): Keyword arguments for the full pricer's price method.
        """
        if not isinstance(num_holdings, int) or num_holdings <= 0:
                raise ValueError("num_holdings must be a positive integer.")
        if not instrument_id:
            raise ValueError("instrument_id must be provided for the position.")

        position_detail = {
            'instrument_id': instrument_id,
            'product_static': product_static,
            'num_holdings': num_holdings,
            'engine_type': pricing_engine_type.lower()
        }

        if position_detail['engine_type'] == 'tff':
            if direct_tff_config is not None:
                # A TFF model and its configuration are being provided directly as a dictionary
                if not isinstance(direct_tff_config, dict):
                    raise TypeError("direct_tff_config must be a dictionary.")

                model_dict = direct_tff_config.get('model_dict')
                raw_names = direct_tff_config.get('raw_input_names')
                norm_params = direct_tff_config.get('normalization_params')

                # Default option_feature_order to 0 if not explicitly in config
                opt_order = direct_tff_config.get('option_feature_order', 0)

                if model_dict is None or not isinstance(model_dict, dict):
                    raise ValueError("direct_tff_config is missing 'model_dict' or it's not a dictionary.")
                if raw_names is None or not isinstance(raw_names, list):
                    raise ValueError("direct_tff_config is missing 'raw_input_names' or it's not a list.")
                if norm_params is None or not isinstance(norm_params, dict):
                    raise ValueError("direct_tff_config is missing 'normalization_params' or it's not a dictionary.")

                # Deserialize the TFF model itself from model_dict
                try:
                    tff_model_instance = TensorFunctionalForm.from_dict(model_dict)
                except Exception as e:
                    raise ValueError(f"Failed to deserialize TFF model from direct_tff_config['model_dict']: {e}")

                position_detail['pricer_engine'] = tff_model_instance
                position_detail['raw_tff_input_names'] = raw_names
                position_detail['normalization_params'] = norm_params
                position_detail['option_feature_order'] = opt_order

            else:
                # No direct TFF config provided, so use the cache via instrument_id
                if instrument_id not in self.tff_model_cache:
                    raise ValueError(
                        f"TFF model for instrument_id '{instrument_id}' not found in cache. "
                        "Either fit and cache it first, or provide its configuration via 'direct_tff_config'."
                    )

                cached_data = self.tff_model_cache[instrument_id]
                position_detail['pricer_engine'] = cached_data['tff_model']
                position_detail['raw_tff_input_names'] = cached_data['raw_tff_input_names']
                position_detail['normalization_params'] = cached_data['normalization_params']
                position_detail['option_feature_order'] = cached_data['option_feature_order']

        elif position_detail['engine_type'] == 'full':
            if not isinstance(full_pricer_instance, PricerBase):
                raise TypeError("full_pricer_instance must be an instance of PricerBase if engine_type is 'full'.")
            position_detail['pricer_engine'] = full_pricer_instance
            position_detail['full_pricer_kwargs'] = full_pricer_kwargs or {}
        else:
            raise ValueError(f"Unsupported pricing_engine_type: {pricing_engine_type}. Choose 'tff' or 'full'.")

        self.positions.append(position_detail)

    def load_portfolio_from_specs(self, portfolio_specs: list[dict]):
        """
        Loads multiple positions into the portfolio from a list of specifications.

        Each specification in the list is a dictionary that should conform to
        the parameters expected by the `add_position` method.

        Args:
            portfolio_specs (list[dict]): A list of dictionaries, where each dictionary
                                          defines a position to be added.
                                          Expected keys in each dict:
                                            'instrument_id' (str)
                                            'product_static_object' (ProductStaticBase)
                                            'num_holdings' (int)
                                            'pricing_engine_type' (str: 'tff' or 'full')
                                            Optional for 'tff' type:
                                              'direct_tff_config' (dict, containing 'model_dict',
                                                                   'raw_input_names', 'normalization_params',
                                                                   'option_feature_order')
                                            Optional for 'full' type:
                                              'full_pricer_instance' (PricerBase)
                                              'full_pricer_kwargs' (dict)
        """
        if not isinstance(portfolio_specs, list):
            raise TypeError("portfolio_specs must be a list of dictionaries.")

        for i, item_spec in enumerate(portfolio_specs):
            if not isinstance(item_spec, dict):
                raise TypeError(f"Each item in portfolio_specs must be a dictionary. Found type {type(item_spec)} at index {i}.")

            try:
                self.add_position(
                    instrument_id=item_spec['instrument_id'],
                    product_static=item_spec['product_static_object'],
                    num_holdings=item_spec.get('num_holdings', 1), # Default if not specified
                    pricing_engine_type=item_spec.get('pricing_engine_type', 'tff'), # Default

                    direct_tff_config=item_spec.get('direct_tff_config'), # Will be None if key doesn't exist

                    full_pricer_instance=item_spec.get('full_pricer_instance'),
                    full_pricer_kwargs=item_spec.get('full_pricer_kwargs')
                )
            except KeyError as e:
                raise ValueError(f"Missing required key {e} in portfolio_spec at index {i}: {item_spec}")
            except Exception as e:
                raise RuntimeError(f"Error adding position from spec at index {i} ('{item_spec.get('instrument_id', 'Unknown ID')}'): {e}")

        print(f"Successfully loaded {len(self.positions)} positions into the portfolio from specifications.")

    def price_portfolio(self,
                        raw_market_scenarios: np.ndarray,
                        scenario_factor_names: list[str],
                        portfolio_rate_pillar_times: np.ndarray = None
                        ) -> np.ndarray:
        """
        Prices all positions in the portfolio for given market scenarios.

        Args:
            raw_market_scenarios (np.ndarray): 2D array of raw market scenarios
                                               (N_scenarios, N_total_market_factors).
            scenario_factor_names (list[str]): Names of the columns in raw_market_scenarios.
            portfolio_rate_pillar_times (np.ndarray, optional): 1D array of rate pillar times (numeric tenors).
                                                               Required if portfolio contains bonds
                                                               priced with full QuantLibBondPricer or FastBondPricer.

        Returns:
            np.ndarray: 1D array of aggregated portfolio prices for each scenario (N_scenarios,).
        """
        if not self.positions:
            return np.array([])

        num_scenarios = raw_market_scenarios.shape[0]
        portfolio_prices_per_scenario = np.zeros(num_scenarios, dtype=float)

        for position_detail in self.positions:
            pricer_engine = position_detail['pricer_engine']
            engine_type = position_detail['engine_type']
            num_holdings = position_detail['num_holdings']

            instrument_prices_this_instrument = np.zeros(num_scenarios, dtype=float)

            if engine_type == 'tff':
                tff_model: TensorFunctionalForm = pricer_engine
                # These are the names of the RAW factors the TFF was originally trained on.
                raw_tff_input_factor_names_for_this_tff = position_detail['raw_tff_input_names']
                norm_params = position_detail['normalization_params']
                opt_feat_order = position_detail['option_feature_order']

                # Select the relevant columns from the global raw_market_scenarios
                try:
                    indices_of_raw_factors_in_global_scenarios = [scenario_factor_names.index(name) for name in raw_tff_input_factor_names_for_this_tff]
                except ValueError as e:
                    raise ValueError(f"A TFF input name in {raw_tff_input_factor_names_for_this_tff} not found in scenario_factor_names {scenario_factor_names} for instrument_id '{position_detail['instrument_id']}'. Error: {e}")

                current_raw_inputs_for_tff = raw_market_scenarios[:, indices_of_raw_factors_in_global_scenarios]

                inputs_for_tff_evaluation = current_raw_inputs_for_tff # Default

                # opt_feat_order should be available from position_detail if it's an option TFF
                opt_feat_order = position_detail.get('option_feature_order', 0)

                if isinstance(position_detail['product_static'], EuropeanOptionStatic) and \
                   norm_params.get('is_engineered', False): # Check if TFF was trained with engineered features

                    s0_factor_actual_name_port = None
                    vol_factor_actual_name_port = None
                    # These indices are relative to raw_tff_input_factor_names_for_this_tff
                    # and thus to the columns of current_raw_inputs_for_tff
                    s0_idx_in_tff_inputs = -1
                    vol_idx_in_tff_inputs = -1

                    # For an option TFF, raw_tff_input_factor_names_for_this_tff is expected to be [full_s0_name, full_vol_name]
                    if len(raw_tff_input_factor_names_for_this_tff) == 2:
                        for i, name in enumerate(raw_tff_input_factor_names_for_this_tff):
                            if name.upper().endswith("_S0"): # Using suffix matching
                                s0_factor_actual_name_port = name
                                s0_idx_in_tff_inputs = i
                            elif name.upper().endswith("_VOLATILITY") or name.upper().endswith("_VOL"): # Using suffix matching
                                vol_factor_actual_name_port = name
                                vol_idx_in_tff_inputs = i

                        if s0_idx_in_tff_inputs == -1 or vol_idx_in_tff_inputs == -1 or s0_idx_in_tff_inputs == vol_idx_in_tff_inputs:
                            raise ValueError(
                                f"Portfolio pricing: Could not identify distinct S0 and Volatility factors for option "
                                f"'{position_detail['instrument_id']}' from its TFF input names: "
                                f"{raw_tff_input_factor_names_for_this_tff}."
                            )
                    else:
                        raise ValueError(
                            f"Portfolio pricing: Option TFF for '{position_detail['instrument_id']}' with engineered features "
                            f"expects 2 raw input factors (S0, Vol), but "
                            f"TFF was trained on {len(raw_tff_input_factor_names_for_this_tff)} names: "
                            f"{raw_tff_input_factor_names_for_this_tff}."
                        )

                    # current_raw_inputs_for_tff columns are ordered as per raw_tff_input_factor_names_for_this_tff
                    s0_scenarios_raw = current_raw_inputs_for_tff[:, s0_idx_in_tff_inputs]
                    vol_scenarios_raw = current_raw_inputs_for_tff[:, vol_idx_in_tff_inputs]

                    engineered_features_test, _ = engineer_option_features( # Changed name for clarity
                        s0_scenarios_raw, vol_scenarios_raw, order=opt_feat_order
                    )
                    # Use normalization_params['means'] and ['stds'] from the cached TFF model data
                    inputs_for_tff_evaluation, _, _ = normalize_features(
                        engineered_features_test, # Changed name
                        norm_params['means'],
                        norm_params['stds']
                    )

                instrument_prices_this_instrument = tff_model(inputs_for_tff_evaluation)

            elif engine_type == 'full':
                full_pricer: PricerBase = pricer_engine
                pricer_kwargs = position_detail.get('full_pricer_kwargs', {})

                if isinstance(full_pricer, (QuantLibBondPricer, FastBondPricer)):
                    if portfolio_rate_pillar_times is None:
                        raise ValueError("portfolio_rate_pillar_times must be provided for full QL/Fast bond pricing in portfolio.")

                    num_rate_pillars = len(portfolio_rate_pillar_times)
                    # Select rate columns from raw_market_scenarios
                    rate_indices = []
                    try:
                        # Attempt to find by name if portfolio_rate_pillar_times contains names (should be numeric tenors)
                        # For simplicity, assume scenario_factor_names aligns with rate_pillar_names for the rate part
                        rate_indices = [scenario_factor_names.index(name) for name in portfolio_rate_pillar_times if isinstance(name, str) and name in scenario_factor_names]
                        if len(rate_indices) != num_rate_pillars: # Fallback if names don't match
                             rate_indices = list(range(num_rate_pillars))
                    except (ValueError, IndexError, TypeError):
                        # Fallback if portfolio_rate_pillar_times are not strings or not found
                        rate_indices = list(range(num_rate_pillars))

                    market_data_for_bond_pricer = raw_market_scenarios[:, rate_indices]

                    if isinstance(full_pricer, QuantLibBondPricer) and full_pricer.is_convertible and 'S0' in scenario_factor_names:
                        s0_col_idx = scenario_factor_names.index('S0')
                        market_data_for_bond_pricer = np.hstack((
                            market_data_for_bond_pricer,
                            raw_market_scenarios[:, s0_col_idx, np.newaxis]
                        ))

                    instrument_prices_this_instrument = full_pricer.price(
                        pillar_times=portfolio_rate_pillar_times, # These are the numeric tenors
                        market_scenario_data=market_data_for_bond_pricer,
                        **pricer_kwargs
                    )
                elif isinstance(full_pricer, BlackScholesPricer):
                    s0_idx = scenario_factor_names.index('S0')
                    vol_idx = scenario_factor_names.index('Volatility')
                    s0_scens = raw_market_scenarios[:, s0_idx]
                    vol_scens = raw_market_scenarios[:, vol_idx]
                    instrument_prices_this_instrument = full_pricer.price(
                        stock_price=s0_scens,
                        volatility=vol_scens,
                        **pricer_kwargs
                    )
                else:
                    raise TypeError(f"Unsupported full pricer type in portfolio: {type(full_pricer)} for instrument_id '{position_detail['instrument_id']}'")
            else:
                raise ValueError(f"Unknown engine_type: {engine_type} for instrument_id '{position_detail['instrument_id']}'")

            # Ensure consistent shapes for aggregation
            if instrument_prices_this_instrument.ndim == 0:
                instrument_prices_this_instrument = np.full(num_scenarios, float(instrument_prices_this_instrument))
            elif len(instrument_prices_this_instrument) != num_scenarios:
                 instrument_prices_this_instrument = instrument_prices_this_instrument.flatten()
                 if len(instrument_prices_this_instrument) != num_scenarios:
                    raise ValueError(f"Price array shape mismatch for instrument '{position_detail['instrument_id']}'. Expected ({num_scenarios},), got {instrument_prices_this_instrument.shape}")

            portfolio_prices_per_scenario += instrument_prices_this_instrument * num_holdings

        return portfolio_prices_per_scenario




In [111]:
# workflow_manager.py
"""
Contains classes to manage the workflow of instrument processing (TFF calibration),
portfolio construction, and portfolio analytics (e.g., VaR).
Uses targeted scenario generation for TFF fitting.
Correctly handles convertible bond TFF factor configurations and pricer parameters.
"""
import json
from datetime import date, datetime
import numpy as np
import QuantLib as ql
import time
import os

# --- JSON Serialization Helpers ---
def portfolio_json_serializer(obj):
    if isinstance(obj, (datetime, date)): return obj.isoformat()
    if isinstance(obj, np.ndarray): return obj.tolist()
    if hasattr(obj, 'to_dict') and callable(obj.to_dict): return obj.to_dict()
    if isinstance(obj, ProductStaticBase): return obj.to_dict()
    if isinstance(obj, ql.Date): return date(obj.year(), obj.month(), obj.day()).isoformat()
    if isinstance(obj, ql.Calendar): return obj.name()
    if isinstance(obj, ql.DayCounter): return obj.name()
    if isinstance(obj, (np.float32, np.float64)): return float(obj)
    if isinstance(obj, (np.int32, np.int64)): return int(obj)
    if isinstance(obj, dict):
        return {k: portfolio_json_serializer(v) for k, v in obj.items()}
    raise TypeError(f"Object of type {obj.__class__.__name__} ({obj}) is not JSON serializable by custom serializer")

def reconstruct_product_static(product_dict: dict) -> ProductStaticBase:
    product_type = product_dict.get('product_type')
    if not product_type:
        raise ValueError("Product dictionary must contain a 'product_type' field.")
    if product_type == 'VanillaBond':
        return QuantLibBondStaticBase.from_dict(product_dict)
    elif product_type == 'CallableBond':
        return CallableBondStaticBase.from_dict(product_dict)
    elif product_type == 'ConvertibleBond':
        return ConvertibleBondStaticBase.from_dict(product_dict)
    elif product_type == 'EuropeanOption':
        return EuropeanOptionStatic.from_dict(product_dict)
    else:
        raise ValueError(f"Unknown product_type for reconstruction: {product_type}")

# --- Utility Function for Step 4 Simplification ---
def generate_portfolio_specs_for_serialization(
    holdings_data: list[dict],
    model_registry: dict,
    instrument_definitions_data_for_pricer_params: list[dict] = None
    ) -> list[dict]:
    portfolio_specs_for_json = []
    if instrument_definitions_data_for_pricer_params is None:
        instrument_definitions_data_for_pricer_params = []

    for holding in holdings_data:
        instrument_id = holding.get("instrument_id")
        if not instrument_id:
            print(f"   Skipping holding due to missing instrument_id: {holding}")
            continue

        if instrument_id in model_registry and not model_registry[instrument_id].get('error'):
            entry = model_registry[instrument_id]
            spec_item = {
                "client_id": holding.get("client_id"),
                "instrument_id": instrument_id,
                "num_holdings": holding.get("num_holdings"),
                "pricing_engine_type": entry["pricing_method"].lower(),
                "product_static_object": entry["product_static_dict"]
            }
            if entry["pricing_method"] == 'TFF' and 'tff_model_dict' in entry:
                spec_item["direct_tff_config"] = {
                    "model_dict": entry["tff_model_dict"],
                    "raw_input_names": entry["tff_raw_input_names"],
                    "normalization_params": entry["tff_normalization_params"],
                    "option_feature_order": entry.get("tff_option_feature_order", 0)
                }
                if 'tff_fixed_pricer_params' in entry:
                    spec_item['pricer_params'] = entry['tff_fixed_pricer_params']

            if entry["pricing_method"] == 'FULL':
                if 'pricer_params' in entry:
                     spec_item['pricer_params'] = entry['pricer_params']
                else:
                    original_instrument_spec = next(
                        (item for item in instrument_definitions_data_for_pricer_params
                         if item.get("instrument_id") == instrument_id),
                        None
                    )
                    if original_instrument_spec and 'pricer_params' in original_instrument_spec:
                         spec_item['pricer_params'] = original_instrument_spec['pricer_params']

            portfolio_specs_for_json.append(spec_item)
        else:
            print(f"   Skipping instrument '{instrument_id}' for JSON spec generation: not in valid model_registry or had an error.")
    return portfolio_specs_for_json


# --- Workflow Classes ---
class InstrumentProcessor:
    def __init__(self, scenario_generator: SimpleRandomScenarioGenerator,
                 global_valuation_date: date,
                 default_numeric_rate_tenors: np.ndarray = None,
                 default_g2_params = None,
                 default_bs_risk_free_rate: float = 0.025,
                 default_bs_dividend_yield: float = 0.0,
                 parallel_workers_tff: int = None,
                 n_scenarios_for_tff_domain: int = 1000
                 ):
        self.scenario_generator = scenario_generator
        self.global_valuation_date = global_valuation_date
        self.default_numeric_rate_tenors = default_numeric_rate_tenors
        self.default_g2_params = default_g2_params
        self.default_bs_risk_free_rate = default_bs_risk_free_rate
        self.default_bs_dividend_yield = default_bs_dividend_yield
        self.parallel_workers_tff = parallel_workers_tff if parallel_workers_tff is not None else (os.cpu_count() if os.cpu_count() else 1)
        self.n_scenarios_for_tff_domain = n_scenarios_for_tff_domain
        self.model_registry = {}

    def _create_pricer_template(self, product_static: ProductStaticBase, instrument_spec: dict):
        pricer_params = instrument_spec.get('pricer_params', {})
        if isinstance(product_static, EuropeanOptionStatic):
            rfr = pricer_params.get('bs_risk_free_rate', self.default_bs_risk_free_rate)
            div = pricer_params.get('bs_dividend_yield', self.default_bs_dividend_yield)
            return BlackScholesPricer(product_static, risk_free_rate=rfr, dividend_yield=div)
        elif isinstance(product_static, CallableBondStaticBase):
            grid_steps = pricer_params.get('g2_grid_steps', 32)
            return QuantLibBondPricer(product_static, method='g2', grid_steps=grid_steps)
        elif isinstance(product_static, ConvertibleBondStaticBase):
            engine_steps = pricer_params.get('conv_engine_steps', 50)
            return QuantLibBondPricer(product_static, method='convertible_binomial', convertible_engine_steps=engine_steps)
        elif isinstance(product_static, QuantLibBondStaticBase):
            if instrument_spec.get('pricer_type_preference', 'QuantLib').upper() == 'FAST':
                 return FastBondPricer(product_static)
            return QuantLibBondPricer(product_static, method='discount')
        else:
            raise ValueError(f"Unsupported product type for pricer template: {type(product_static)}")

    def _get_scenario_slice(self, all_scenarios, all_factor_names, target_factor_names_for_tff):
        if not target_factor_names_for_tff:
            return np.array([]).reshape(all_scenarios.shape[0],0)
        try:
            global_indices_map = {name: i for i, name in enumerate(all_factor_names)}
            ordered_indices = [global_indices_map[name] for name in target_factor_names_for_tff]
            return all_scenarios[:, ordered_indices]
        except KeyError as e:
            missing_factor = str(e).strip("'")
            raise ValueError(
                f"Error slicing scenarios: Factor name '{missing_factor}' required by TFF "
                f"not found in generated scenario factor names. "
                f"Required by TFF: {target_factor_names_for_tff}, "
                f"Available from generator: {all_factor_names}."
            )
        except Exception as e:
            raise RuntimeError(f"General error during scenario slicing for TFF factors {target_factor_names_for_tff}: {e}")

    def process_instruments(self, instrument_definitions: list[dict],
                            global_market_scenarios: np.ndarray,
                            global_factor_names: list[str]):
        print(f"Processing {len(instrument_definitions)} instrument definitions...")
        ql_val_date_obj = ql.Date(self.global_valuation_date.day, self.global_valuation_date.month, self.global_valuation_date.year)
        ql.Settings.instance().evaluationDate = ql_val_date_obj

        for spec_idx, instrument_spec in enumerate(instrument_definitions):
            instrument_id = instrument_spec.get('instrument_id')
            product_type_str = instrument_spec.get('product_type')
            params = instrument_spec.get('params', {})
            pricing_preference = instrument_spec.get('pricing_preference', 'FULL').upper()

            if not instrument_id or not product_type_str or not params:
                print(f"  Skipping instrument definition at index {spec_idx} due to missing id, type, or params.")
                continue

            print(f"  Processing instrument: {instrument_id} (Type: {product_type_str}, Preference: {pricing_preference})")

            if 'valuation_date' not in params: params['valuation_date'] = self.global_valuation_date
            if 'product_type' not in params: params['product_type'] = product_type_str

            try:
                product_static_object = reconstruct_product_static(params)
            except Exception as e:
                print(f"    ERROR creating product static for {instrument_id}: {e}")
                self.model_registry[instrument_id] = {'error': str(e), 'pricing_method': 'ERROR', 'product_static_dict': params}
                continue

            pricer_template = self._create_pricer_template(product_static_object, instrument_spec)

            registry_entry = {
                'instrument_id': instrument_id,
                'product_static_dict': product_static_object.to_dict(),
                'pricing_method': pricing_preference
            }
            if 'pricer_params' in instrument_spec:
                registry_entry['pricer_params'] = instrument_spec['pricer_params']


            if pricing_preference == 'TFF':
                tff_config = instrument_spec.get('tff_config', {})
                n_train = tff_config.get('n_train', 64)
                n_test = tff_config.get('n_test', 10)
                option_feature_order_for_tff = tff_config.get('option_feature_order', 0)

                conv_market_inputs_dynamic_flag = False
                if isinstance(product_static_object, ConvertibleBondStaticBase):
                    conv_market_inputs_dynamic_flag = tff_config.get('convertible_tff_market_inputs_as_factors', False)

                extra_pricer_args_for_tff_training = {}
                if isinstance(product_static_object, CallableBondStaticBase):
                    g2_p_spec = instrument_spec.get('pricer_params', {}).get('g2_params', self.default_g2_params)
                    if g2_p_spec: extra_pricer_args_for_tff_training['g2_params'] = g2_p_spec

                if isinstance(product_static_object, ConvertibleBondStaticBase) and not conv_market_inputs_dynamic_flag:
                    fixed_conv_params = instrument_spec.get('pricer_params', {})
                    # For this case (CB TFF with S0 dynamic but others fixed),
                    # TFFCalibrate will add S0 as a factor. We provide the rest as fixed.
                    # The pricer expects 's0_val', 'dividend_yield', etc.
                    extra_pricer_args_for_tff_training.update({
                        # S0 is dynamic, so 's0_val' is NOT passed here as fixed.
                        # It will come from the scenario data for the TFF.
                        'dividend_yield': fixed_conv_params.get('dividend_yield'),
                        'equity_volatility': fixed_conv_params.get('equity_volatility'),
                        'credit_spread': fixed_conv_params.get('credit_spread')
                    })
                    # Validate that these necessary fixed params are present
                    required_fixed_for_cb_tff_partial_dynamic = ['dividend_yield', 'equity_volatility', 'credit_spread']
                    missing_fixed = [k for k in required_fixed_for_cb_tff_partial_dynamic if extra_pricer_args_for_tff_training.get(k) is None]
                    if missing_fixed:
                        error_msg = (f"Missing fixed pricer_params for CB TFF {instrument_id} (S0 dynamic, others fixed). "
                                     f"Needed: {missing_fixed}. Got from pricer_params: {fixed_conv_params}")
                        print(f"    WARNING: {error_msg}")
                        registry_entry['pricing_method'] = 'FULL'
                        registry_entry['error_tff_calibration'] = error_msg
                        self.model_registry[instrument_id] = registry_entry
                        continue


                try:
                    print(f"    Calibrating TFF for {instrument_id}...")
                    tff_calibrator = TensorFunctionalFormCalibrate(
                        pricer_template=pricer_template,
                        scenario_generator=self.scenario_generator,
                        bond_numeric_tenors=self.default_numeric_rate_tenors,
                        # Corrected keyword argument:
                        convertible_tff_market_inputs_as_factors=conv_market_inputs_dynamic_flag
                    )

                    target_factors_for_tff = tff_calibrator.tff_input_raw_factor_names
                    scenarios_for_this_tff = self._get_scenario_slice(global_market_scenarios, global_factor_names, target_factors_for_tff)

                    if scenarios_for_this_tff.size == 0 and target_factors_for_tff:
                         print(f"   WARNING: Scenario slice is empty for TFF {instrument_id} with expected factors {target_factors_for_tff}. Skipping fitting.")
                         raise ValueError("Empty scenario slice for TFF fitting.")

                    s_t = time.time()
                    model_tff, _, _, rmse_tff, norm_params_tff = tff_calibrator.sample_and_fit(
                        full_market_scenarios_for_tff_factors=scenarios_for_this_tff,
                        n_train=n_train, n_test=n_test, random_seed=instrument_spec.get('seed', 42),
                        parallel_workers=self.parallel_workers_tff,
                        option_feature_order=option_feature_order_for_tff,
                        **extra_pricer_args_for_tff_training
                    )
                    fit_time = time.time() - s_t

                    if model_tff and norm_params_tff:
                        registry_entry['tff_model_dict'] = model_tff.to_dict()
                        registry_entry['tff_raw_input_names'] = target_factors_for_tff
                        registry_entry['tff_normalization_params'] = norm_params_tff
                        registry_entry['tff_option_feature_order'] = option_feature_order_for_tff
                        registry_entry['tff_rmse'] = rmse_tff
                        registry_entry['tff_fit_time_seconds'] = fit_time
                        if isinstance(product_static_object, ConvertibleBondStaticBase) and not conv_market_inputs_dynamic_flag:
                            registry_entry['tff_fixed_pricer_params'] = extra_pricer_args_for_tff_training

                        print(f"      TFF for {instrument_id} fitted. RMSE: {rmse_tff:.6f}, Time: {fit_time:.2f}s")
                    else:
                        raise RuntimeError("TFF fitting returned None for model or norm_params.")

                except Exception as e:
                    print(f"    ERROR during TFF calibration for {instrument_id}: {e}")
                    registry_entry['pricing_method'] = 'FULL'
                    registry_entry['error_tff_calibration'] = str(e)

            self.model_registry[instrument_id] = registry_entry
        print("Finished processing instrument definitions.")
        return self.model_registry

    def save_model_registry(self, filepath: str):
        print(f"Saving model registry to {filepath}...")
        try:
            with open(filepath, 'w') as f:
                json.dump(self.model_registry, f, indent=4, default=portfolio_json_serializer)
            print("  Model registry saved successfully.")
        except Exception as e:
            print(f"  ERROR saving model registry: {e}")

    @classmethod
    def load_model_registry(cls, filepath: str) -> dict:
        print(f"Loading model registry from {filepath}...")
        try:
            with open(filepath, 'r') as f:
                registry = json.load(f)
            print("  Model registry loaded successfully.")
            return registry
        except Exception as e:
            print(f"  ERROR loading model registry: {e}")
            return {}


class PortfolioBuilder:
    def __init__(self, model_registry: dict):
        if not isinstance(model_registry, dict):
            raise TypeError("model_registry must be a dictionary.")
        self.model_registry = model_registry
        self.uncalculated_instruments = []

    def build_portfolios_from_specs(self, portfolio_specs_list: list[dict],
                                       global_valuation_date: date,
                                       default_g2_params=None,
                                       default_bs_rfr=0.025, default_bs_div=0.0
                                       ) -> dict[str, Portfolio]:
        print(f"Building portfolios from {len(portfolio_specs_list)} detailed specifications...")
        portfolios = {}
        self.uncalculated_instruments = []

        for spec_idx, spec in enumerate(portfolio_specs_list):
            client_id = spec.get('client_id')
            instrument_id = spec.get('instrument_id')
            num_holdings = spec.get('num_holdings')
            product_static_dict_from_spec = spec.get('product_static_object')
            pricing_method_from_spec = spec.get('pricing_engine_type', 'full').lower()
            direct_tff_config_from_spec = spec.get('direct_tff_config')
            pricer_params_from_spec = spec.get('pricer_params', {})

            if not client_id or not instrument_id or num_holdings is None or product_static_dict_from_spec is None:
                print(f"  Skipping spec at index {spec_idx}: missing essential fields.")
                continue

            try:
                if 'valuation_date' not in product_static_dict_from_spec:
                    product_static_dict_from_spec['valuation_date'] = global_valuation_date
                product_static_object = reconstruct_product_static(product_static_dict_from_spec)
            except Exception as e:
                print(f"  ERROR reconstructing product static for '{instrument_id}' from spec: {e}. Skipping.")
                if instrument_id not in self.uncalculated_instruments: self.uncalculated_instruments.append(instrument_id)
                continue

            if client_id not in portfolios: portfolios[client_id] = Portfolio()
            portfolio_instance = portfolios[client_id]

            final_pricing_method = pricing_method_from_spec
            final_direct_tff_config = direct_tff_config_from_spec
            final_full_pricer_instance = None
            final_full_pricer_kwargs = {}

            if final_pricing_method == 'tff':
                if not final_direct_tff_config:
                    if instrument_id in self.model_registry and self.model_registry[instrument_id].get('pricing_method', '').upper() == 'TFF':
                        entry = self.model_registry[instrument_id]
                        if all(k in entry for k in ['tff_model_dict', 'tff_raw_input_names', 'tff_normalization_params']):
                            final_direct_tff_config = {
                                'model_dict': entry['tff_model_dict'],
                                'raw_input_names': entry['tff_raw_input_names'],
                                'normalization_params': entry['tff_normalization_params'],
                                'option_feature_order': entry.get('tff_option_feature_order', 0)
                            }
                            if 'tff_fixed_pricer_params' in entry:
                                final_full_pricer_kwargs.update(entry['tff_fixed_pricer_params'])
                        else:
                            print(f"  WARNING: TFF data incomplete for '{instrument_id}' in registry. Fallback to FULL.")
                            final_pricing_method = 'full'
                    else:
                        print(f"  WARNING: TFF spec for '{instrument_id}' missing direct_tff_config and not found as TFF in registry. Fallback to FULL.")
                        final_pricing_method = 'full'
                # If direct_tff_config was provided in the spec, ensure any fixed params are also there
                # These fixed params would have been part of the TFF training context.
                if final_direct_tff_config and isinstance(product_static_object, ConvertibleBondStaticBase):
                     # pricer_params_from_spec might contain the fixed CB params for this direct TFF
                     final_full_pricer_kwargs.update(pricer_params_from_spec)


            if final_pricing_method == 'full':
                current_pricer_params = pricer_params_from_spec
                try:
                    if isinstance(product_static_object, EuropeanOptionStatic):
                        rfr = current_pricer_params.get('bs_risk_free_rate', default_bs_rfr)
                        div = current_pricer_params.get('bs_dividend_yield', default_bs_div)
                        final_full_pricer_instance = BlackScholesPricer(product_static_object, rfr, div)
                    elif isinstance(product_static_object, CallableBondStaticBase):
                        grid_steps = current_pricer_params.get('g2_grid_steps', 32)
                        final_full_pricer_instance = QuantLibBondPricer(product_static_object, method='g2', grid_steps=grid_steps)
                        final_full_pricer_kwargs = {'g2_params': current_pricer_params.get('g2_params', default_g2_params)}
                    elif isinstance(product_static_object, ConvertibleBondStaticBase):
                        engine_steps = current_pricer_params.get('conv_engine_steps', 50)
                        final_full_pricer_instance = QuantLibBondPricer(product_static_object, method='convertible_binomial', convertible_engine_steps=engine_steps)
                        # For FULL pricing of CB, pricer needs dynamic params from kwargs
                        # These should be in current_pricer_params
                        cb_full_kwargs = {
                            's0_val': current_pricer_params.get('s0_val', current_pricer_params.get('initial_stock_price')),
                            'dividend_yield': current_pricer_params.get('dividend_yield'),
                            'equity_volatility': current_pricer_params.get('equity_volatility'),
                            'credit_spread': current_pricer_params.get('credit_spread')
                        }
                        if any(val is None for val in cb_full_kwargs.values()):
                            # Attempt to get from model_registry if not in spec's pricer_params
                            if instrument_id in self.model_registry and 'pricer_params' in self.model_registry[instrument_id]:
                                 reg_pricer_params = self.model_registry[instrument_id]['pricer_params']
                                 for k_cb_orig, k_cb_pricer in [('initial_stock_price','s0_val'), ('dividend_yield','dividend_yield'),
                                                                ('equity_volatility','equity_volatility'), ('credit_spread','credit_spread')]:
                                     if cb_full_kwargs.get(k_cb_pricer) is None and reg_pricer_params.get(k_cb_orig) is not None:
                                         cb_full_kwargs[k_cb_pricer] = reg_pricer_params[k_cb_orig]
                            if any(val is None for val in cb_full_kwargs.values()):
                                raise ValueError(f"Missing required pricer_params for FULL CB pricing of {instrument_id}. Needed: s0_val, dividend_yield, equity_volatility, credit_spread. Got: {cb_full_kwargs}")
                        final_full_pricer_kwargs.update(cb_full_kwargs)

                    elif isinstance(product_static_object, QuantLibBondStaticBase):
                        final_full_pricer_instance = QuantLibBondPricer(product_static_object, method='discount')
                    else:
                        raise ValueError("Unknown product type for full pricer reconstruction.")
                except Exception as e_pricer:
                    print(f"  WARNING: Cannot create full pricer for '{instrument_id}': {e_pricer}. Skipping.")
                    if instrument_id not in self.uncalculated_instruments: self.uncalculated_instruments.append(instrument_id)
                    continue

            try:
                portfolio_instance.add_position(
                    instrument_id=instrument_id, product_static=product_static_object,
                    num_holdings=num_holdings, pricing_engine_type=final_pricing_method,
                    direct_tff_config=final_direct_tff_config if final_pricing_method == 'tff' else None,
                    full_pricer_instance=final_full_pricer_instance if final_pricing_method == 'full' else None,
                    full_pricer_kwargs=final_full_pricer_kwargs
                )
            except Exception as e:
                print(f"  ERROR adding position '{instrument_id}' to portfolio for '{client_id}': {e}")
                if instrument_id not in self.uncalculated_instruments: self.uncalculated_instruments.append(instrument_id)

        if self.uncalculated_instruments:
            print(f"  Summary: Uncalculated instruments during build_portfolios_from_specs: {self.uncalculated_instruments}")
        print(f"Finished building {len(portfolios)} portfolios from detailed specs.")
        return portfolios


class PortfolioAnalytics:
    def __init__(self,
                 client_portfolios: dict[str, Portfolio],
                 global_market_scenarios: np.ndarray,
                 global_factor_names: list[str],
                 numeric_rate_tenors: np.ndarray,
                 scenario_generator_for_base_values: SimpleRandomScenarioGenerator
                 ):
        self.client_portfolios = client_portfolios
        self.global_market_scenarios = global_market_scenarios
        self.global_factor_names = global_factor_names
        self.numeric_rate_tenors = numeric_rate_tenors
        self.scenario_generator_for_base_values = scenario_generator_for_base_values
        self.results = {}

    def calculate_base_portfolio_values(self) -> dict[str, float]:
        base_values = {}
        base_value_scenario_list = []
        for factor_name in self.global_factor_names:
            val_found = False
            # Corrected loop variable from 'm' to 'current_map'
            for current_map in [self.scenario_generator_for_base_values.base_rates_map,
                                self.scenario_generator_for_base_values.base_s0_map,
                                self.scenario_generator_for_base_values.base_vol_map]:
                if factor_name in current_map:
                    base_value_scenario_list.append(current_map[factor_name])
                    val_found = True
                    break
            if not val_found:
                sg = self.scenario_generator_for_base_values
                # Check merged s0_map in scenario_generator if it holds other factors
                # This relies on how scenario_gen_global was configured with merged maps
                # A more robust scenario_generator would have a method .get_base_value(factor_name)
                # or distinct maps for all factor types it can generate.
                # For now, we check the maps it's known to have.
                # The error "Missing dividend_yield" suggests this logic might not find all needed base values.
                # If scenario_gen_global.base_s0_map was merged with other_conv_factors_base_map, it should be found.
                if hasattr(sg, 'base_s0_map') and factor_name in sg.base_s0_map:
                     base_value_scenario_list.append(sg.base_s0_map[factor_name])
                     val_found = True
                elif hasattr(sg, 'base_vol_map') and factor_name in sg.base_vol_map:
                     base_value_scenario_list.append(sg.base_vol_map[factor_name])
                     val_found = True
            if not val_found:
                print(f"Warning: Factor '{factor_name}' for base value not found in generator base maps. Using 0.0.")
                base_value_scenario_list.append(0.0)
        base_value_scenario_np = np.array([base_value_scenario_list])

        for client_id, portfolio_obj in self.client_portfolios.items():
            if portfolio_obj.positions:
                try:
                    base_val = portfolio_obj.price_portfolio(
                        raw_market_scenarios=base_value_scenario_np,
                        scenario_factor_names=self.global_factor_names,
                        portfolio_rate_pillar_times=self.numeric_rate_tenors
                    )[0]
                    base_values[client_id] = base_val
                except Exception as e:
                    print(f"  ERROR calculating base value for portfolio {client_id}: {e}")
                    base_values[client_id] = np.nan
            else:
                base_values[client_id] = 0.0
        return base_values

    def run_var_analysis(self, var_percentiles: list[float] = None):
        if var_percentiles is None:
            var_percentiles = [1.0, 5.0]

        print(f"Running VaR Analysis for percentiles: {[f'{(100-p):.0f}%' for p in var_percentiles]}")

        base_portfolio_values = self.calculate_base_portfolio_values()
        self.results = {}

        for client_id, portfolio_obj in self.client_portfolios.items():
            client_results = {'base_value': base_portfolio_values.get(client_id, np.nan)}
            if portfolio_obj.positions and not np.isnan(client_results['base_value']):
                print(f"  Analyzing portfolio for {client_id}...")
                try:
                    portfolio_values_scenarios = portfolio_obj.price_portfolio(
                        raw_market_scenarios=self.global_market_scenarios,
                        scenario_factor_names=self.global_factor_names,
                        portfolio_rate_pillar_times=self.numeric_rate_tenors
                    )

                    client_results['mean_scenario_value'] = np.mean(portfolio_values_scenarios)
                    client_results['std_dev_scenario_value'] = np.std(portfolio_values_scenarios)
                    pnl_distribution = portfolio_values_scenarios - client_results['base_value']
                    client_results['pnl_distribution_mean'] = np.mean(pnl_distribution)
                    client_results['pnl_distribution_std_dev'] = np.std(pnl_distribution)

                    vars_calculated = {}
                    for p in var_percentiles:
                        var_value = np.percentile(pnl_distribution, p)
                        vars_calculated[f"var_{(100-p):.0f}pct"] = -var_value
                    client_results['var_values'] = vars_calculated

                    print(f"    Client {client_id}: Base Value={client_results['base_value']:.2f}, "
                          f"Mean Scen. Value={client_results['mean_scenario_value']:.2f}, "
                          f"VaRs: {vars_calculated}")

                except Exception as e:
                    print(f"    ERROR during VaR analysis for portfolio {client_id}: {e}")
                    client_results['error_var_analysis'] = str(e)
            else:
                if not portfolio_obj.positions: print(f"  Portfolio for {client_id} is empty, skipping VaR.")
                else: print(f"  Base value for portfolio {client_id} could not be calculated (was NaN), skipping VaR.")
                client_results['var_values'] = {f"var_{(100-p):.0f}pct": np.nan for p in var_percentiles}
            self.results[client_id] = client_results
        return self.results



In [114]:
# main_demonstration.py
"""
Main script demonstrating the new workflow using InstrumentProcessor, PortfolioBuilder,
and PortfolioAnalytics.
"""
import QuantLib as ql
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import time
import os
import json

def run_demonstration(
    enable_parallel_tff_fitting: bool = True,
    use_hardcoded_g2_params: bool = True
    ):
    print(f"--- FastRiskDemo with Workflow Manager (Parallel TFF: {enable_parallel_tff_fitting}) ---")

    # --- Global Setup ---
    val_d = date(2025, 5, 18)
    numeric_rate_tenors = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0])
    DEMO_CURRENCY = "USD"
    DEMO_RATE_INDEX_STUB = "IR"
    OPT_UNDERLYING_SYMBOL = "DEMO_OPT_STOCK"
    CONV_UNDERLYING_SYMBOL = "DEMO_CONV_STOCK"

    default_g2_p = (0.01, 0.003, 0.015, 0.006, -0.75) if use_hardcoded_g2_params else None

    base_demo_currency_rates_values = np.array([0.020, 0.021, 0.022, 0.025, 0.027, 0.030, 0.032, 0.033])
    base_rates_map = {f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_{t:.2f}Y": base_demo_currency_rates_values[i] for i, t in enumerate(numeric_rate_tenors)}

    opt_s0_factor = f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_S0"
    opt_vol_factor = f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_VOL"
    conv_s0_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_S0"
    conv_vol_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_EQVOL"
    conv_div_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_DIVYIELD"
    conv_cs_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_CS"

    base_s0_map = {opt_s0_factor: 100.0, conv_s0_factor: 100.0}
    base_vol_map = {opt_vol_factor: 0.25, conv_vol_factor: 0.25}

    base_other_factors_map_for_conv = {
        conv_div_factor: 0.01,
        conv_cs_factor: 0.015
    }
    temp_s0_map_for_gen = {**base_s0_map, **base_other_factors_map_for_conv}
    temp_vol_map_for_gen = {**base_vol_map}

    scenario_gen_global = SimpleRandomScenarioGenerator(
        base_rates_map=base_rates_map,
        base_s0_map=temp_s0_map_for_gen,
        base_vol_map=temp_vol_map_for_gen,
        random_seed=42
    )
    N_GLOBAL_SCENARIOS = 1000
    global_market_scenarios, global_factor_names = scenario_gen_global.generate_scenarios(N_GLOBAL_SCENARIOS)
    print(f"Generated {N_GLOBAL_SCENARIOS} global market scenarios with factors: {global_factor_names}")


    # --- Step 1: Instrument Processing and TFF Calibration ---
    print("\\n--- Step 1: Instrument Processing & TFF Calibration ---")

    default_conv_fixed_pricer_params_for_tff_training = {
        # These are used by the pricer if these market inputs are NOT TFF factors
        # The QuantLibBondPricer expects keys like 's0_val', 'dividend_yield', etc.
        's0_val': 100.0,
        'dividend_yield': 0.01,
        'equity_volatility': 0.25,
        'credit_spread': 0.015
    }

    instrument_definitions_data = [
        {
            "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_VANILLA_5Y",
            "product_type": "VanillaBond", "pricing_preference": "TFF",
            "params": { "valuation_date": val_d, "maturity_date": (val_d + relativedelta(years=5)),
                        "coupon_rate": 0.03, "face_value": 100.0, "currency": DEMO_CURRENCY,
                        "index_stub": DEMO_RATE_INDEX_STUB, "freq": 2, "settlement_days": 0 },
            "tff_config": {"n_train": 64, "n_test": 10}
        },
        {
            "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_CALLABLE_5Y_G2",
            "product_type": "CallableBond", "pricing_preference": "TFF",
            "params": { "valuation_date": val_d, "maturity_date": (val_d + relativedelta(years=5)),
                        "coupon_rate": 0.032, "face_value": 100.0, "currency": DEMO_CURRENCY,
                        "index_stub": DEMO_RATE_INDEX_STUB, "freq": 2,
                        "call_dates": [(val_d + relativedelta(years=y)) for y in [2,3,4]],
                        "call_prices": [102.0, 101.0, 100.0]},
            "pricer_params": {"g2_params": default_g2_p},
            "tff_config": {"n_train": 64, "n_test": 10}
        },
        {
            "instrument_id": f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_CONV_BOND_5Y_S0_DYNAMIC",
            "product_type": "ConvertibleBond", "pricing_preference": "TFF",
            "params": {
                'valuation_date': val_d, 'issue_date': (val_d - relativedelta(months=6)),
                'maturity_date': (val_d + relativedelta(years=5, months=-6)), 'coupon_rate': 0.02,
                'conversion_ratio': 20.0, 'face_value': 100.0, 'currency': DEMO_CURRENCY,
                'index_stub': DEMO_RATE_INDEX_STUB, 'underlying_symbol': CONV_UNDERLYING_SYMBOL, 'freq': 2
            },
            "pricer_params": default_conv_fixed_pricer_params_for_tff_training,
            "tff_config": {"n_train": 128, "n_test": 10,
                           "convertible_tff_market_inputs_as_factors": False
                          }
        },
        {
            "instrument_id": f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_EURO_CALL_1Y_STRIKE105_ORD2",
            "product_type": "EuropeanOption", "pricing_preference": "TFF",
            "params": { 'valuation_date': val_d, 'expiry_date': (val_d + relativedelta(years=1)),
                        'strike_price': 105.0, 'option_type': 'call',
                        'currency': DEMO_CURRENCY, 'underlying_symbol': OPT_UNDERLYING_SYMBOL,
            }, "pricer_params": { 'bs_risk_free_rate': 0.025, 'bs_dividend_yield': 0.01 },
            "tff_config": {"n_train": 128, "n_test": 10, "option_feature_order": 2}
        },
        { "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_VANILLA_10Y_FULL",
          "product_type": "VanillaBond", "pricing_preference": "FULL",
          "params": { "valuation_date": val_d, "maturity_date": (val_d + relativedelta(years=10)),
                      "coupon_rate": 0.035, "face_value": 100.0, "currency": DEMO_CURRENCY,
                      "index_stub": DEMO_RATE_INDEX_STUB, "freq": 2 }}
    ]

    instrument_processor = InstrumentProcessor(
        scenario_generator=scenario_gen_global, global_valuation_date=val_d,
        default_numeric_rate_tenors=numeric_rate_tenors, default_g2_params=default_g2_p,
        default_bs_risk_free_rate=0.025, default_bs_dividend_yield=0.01,
        parallel_workers_tff=os.cpu_count() if enable_parallel_tff_fitting else False,
        n_scenarios_for_tff_domain=500 )

    model_registry = instrument_processor.process_instruments(
        instrument_definitions_data, global_market_scenarios, global_factor_names )

    instrument_processor.save_model_registry("model_registry.json")

    # --- Step 2: Portfolio Construction ---
    print("\\n--- Step 2: Portfolio Construction ---")
    holdings_data = [
        {"client_id": "ClientA", "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_VANILLA_5Y", "num_holdings": 100},
        {"client_id": "ClientA", "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_CALLABLE_5Y_G2", "num_holdings": 50},
        {"client_id": "ClientB", "instrument_id": f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_CONV_BOND_5Y_S0_DYNAMIC", "num_holdings": 35},
        {"client_id": "ClientB", "instrument_id": f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_CONV_BOND_5Y_ALL_DYNAMIC", "num_holdings": 40},
        {"client_id": "ClientB", "instrument_id": f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_EURO_CALL_1Y_STRIKE105_ORD2", "num_holdings": 200},
        {"client_id": "ClientA", "instrument_id": f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_VANILLA_10Y_FULL", "num_holdings": 80},
        {"client_id": "ClientA", "instrument_id": "MISSING_INSTRUMENT_ID_EXAMPLE", "num_holdings": 10}
    ]

    initial_portfolio_specs = generate_portfolio_specs_for_serialization(
        holdings_data=holdings_data, model_registry=model_registry,
        instrument_definitions_data_for_pricer_params=instrument_definitions_data )

    portfolio_builder_initial = PortfolioBuilder(model_registry)
    client_portfolios = portfolio_builder_initial.build_portfolios_from_specs(
        portfolio_specs_list=initial_portfolio_specs, global_valuation_date=val_d,
        default_g2_params=default_g2_p, default_bs_rfr=0.025, default_bs_div=0.01 )

    if portfolio_builder_initial.uncalculated_instruments:
        print(f"  WARNING (Initial Build): Uncalculated instruments: {portfolio_builder_initial.uncalculated_instruments}")

    # --- Step 3: Portfolio Pricing / VaR Calculation (using PortfolioAnalytics) ---
    print("\\n--- Step 3: Portfolio Pricing / VaR (using PortfolioAnalytics) ---")
    if client_portfolios:
        portfolio_analyzer = PortfolioAnalytics(
            client_portfolios=client_portfolios, global_market_scenarios=global_market_scenarios,
            global_factor_names=global_factor_names, numeric_rate_tenors=numeric_rate_tenors,
            scenario_generator_for_base_values=scenario_gen_global )
        var_results = portfolio_analyzer.run_var_analysis(var_percentiles=[1.0, 5.0])
    else: print("  No client portfolios were built, skipping VaR analysis.")

    # --- Step 4: JSON Serialization/Deserialization Demo for Portfolio Specs ---
    print("\\n--- Step 4: Portfolio JSON Serialization/Deserialization Demo ---")
    portfolio_json_string = None
    if initial_portfolio_specs:
        try:
            portfolio_json_string = json.dumps(initial_portfolio_specs, indent=4, default=portfolio_json_serializer)
            print("\\n--- Portfolio Specifications (JSON Serialized) ---")
            if initial_portfolio_specs:
                 print("Sample of first item in JSON specs:"); print(json.dumps(initial_portfolio_specs[3], indent=4, default=portfolio_json_serializer))
        except Exception as e: print(f"   ERROR serializing portfolio specs to JSON: {e}"); portfolio_json_string = None
    else: print("\\n--- No valid portfolio specifications to serialize to JSON ---")

    if portfolio_json_string:
        print("\\n--- Loading and Pricing Portfolio from JSON String ---")
        try:
            loaded_portfolio_specs_from_str = json.loads(portfolio_json_string)
            portfolio_builder_from_json = PortfolioBuilder(model_registry)
            client_portfolios_from_json = portfolio_builder_from_json.build_portfolios_from_specs(
                portfolio_specs_list=loaded_portfolio_specs_from_str, global_valuation_date=val_d,
                default_g2_params=default_g2_p, default_bs_rfr=0.025, default_bs_div=0.01 )
            if portfolio_builder_from_json.uncalculated_instruments:
                 print(f"  WARNING (JSON Load): Uncalculated instruments: {portfolio_builder_from_json.uncalculated_instruments}")
            if client_portfolios_from_json:
                reloaded_portfolio_analyzer = PortfolioAnalytics(
                    client_portfolios=client_portfolios_from_json, global_market_scenarios=global_market_scenarios,
                    global_factor_names=global_factor_names, numeric_rate_tenors=numeric_rate_tenors,
                    scenario_generator_for_base_values=scenario_gen_global )
                print("  Results for reloaded portfolio from JSON:")
                reloaded_var_results = reloaded_portfolio_analyzer.run_var_analysis(var_percentiles=[1.0, 5.0])
            else: print("  No client portfolios were built from JSON specs.")
        except Exception as e:
            print(f"   ERROR loading or pricing portfolio from JSON string: {e}")
            import traceback; traceback.print_exc()

    print("\\n--- End of Demonstration ---")

if __name__ == "__main__":
    try:
        print(f"QuantLib version: {ql.__version__}")
        run_demonstration( enable_parallel_tff_fitting=False )
    except NameError as e:
        if any(cn in str(e) for cn in ['ProductStaticBase','InstrumentProcessor','PortfolioBuilder','PortfolioAnalytics']):
            print(f"ERROR: Class not defined. Ensure all notebook cells for class definitions are executed. Details: {e}")
        elif 'QuantLib' in str(e) or 'ql' in str(e): print("ERROR: QuantLib not found/imported.")
        else: print(f"A NameError: {e}"); import traceback; traceback.print_exc()
    except Exception as e:
        print(f"An unexpected error: {e}"); import traceback; traceback.print_exc()


QuantLib version: 1.38
--- FastRiskDemo with Workflow Manager (Parallel TFF: False) ---
Generated 1000 global market scenarios with factors: ['USD_DEMO_CONV_STOCK_CS', 'USD_DEMO_CONV_STOCK_DIVYIELD', 'USD_DEMO_CONV_STOCK_EQVOL', 'USD_DEMO_CONV_STOCK_S0', 'USD_DEMO_OPT_STOCK_S0', 'USD_DEMO_OPT_STOCK_VOL', 'USD_IR_0.25Y', 'USD_IR_0.50Y', 'USD_IR_1.00Y', 'USD_IR_10.00Y', 'USD_IR_2.00Y', 'USD_IR_3.00Y', 'USD_IR_5.00Y', 'USD_IR_7.00Y']
\n--- Step 1: Instrument Processing & TFF Calibration ---
Processing 5 instrument definitions...
  Processing instrument: USD_IR_VANILLA_5Y (Type: VanillaBond, Preference: TFF)
    Calibrating TFF for USD_IR_VANILLA_5Y...
   Generating 64 training prices sequentially...
   Generating 10 test prices sequentially...
      TFF for USD_IR_VANILLA_5Y fitted. RMSE: 0.000006, Time: 0.02s
  Processing instrument: USD_IR_CALLABLE_5Y_G2 (Type: CallableBond, Preference: TFF)
    Calibrating TFF for USD_IR_CALLABLE_5Y_G2...
   Generating 64 training prices sequentially..

In [129]:
# tff_performance_demo.py
"""
Demonstrates and compares the performance of Tensor Functional Form (TFF) models
against full pricers for various financial instruments.
Includes TFF calibration time in the performance summary.
"""
import QuantLib as ql
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import os

def run_performance_demonstration(
    n_benchmark_scenarios: int = 5000,
    n_tff_train_samples: int = 256,
    n_tff_test_samples: int = 50,
    enable_parallel_tff_fitting: bool = True,
    use_hardcoded_g2_params: bool = True,
    option_feature_eng_order: int = 2
    ):
    """
    Runs the performance comparison between TFF and full pricers.
    """
    print(f"--- TFF Performance Demonstration ---")
    print(f"Benchmarking with {n_benchmark_scenarios} scenarios.")
    print(f"TFF training with {n_tff_train_samples} samples, testing with {n_tff_test_samples} samples.") # Updated print

    # --- Helper function to get scenario slices ---
    def get_scenario_slice(all_scenarios, all_factor_names, target_factor_names_for_tff):
        if not target_factor_names_for_tff:
            return np.array([]).reshape(all_scenarios.shape[0],0)
        try:
            global_indices_map = {name: i for i, name in enumerate(all_factor_names)}
            ordered_indices = [global_indices_map[name] for name in target_factor_names_for_tff]
            return all_scenarios[:, ordered_indices]
        except KeyError as e:
            missing_factor = str(e).strip("'")
            raise ValueError(
                f"Error slicing scenarios: Factor name '{missing_factor}' required by TFF "
                f"not found in generated scenario factor names. "
                f"Required by TFF: {target_factor_names_for_tff}, "
                f"Available from generator: {all_factor_names}."
            )
        except Exception as e:
            raise RuntimeError(f"General error during scenario slicing for TFF factors {target_factor_names_for_tff}: {e}")

    # --- Global Setup ---
    val_d = date(2025, 5, 18)
    numeric_rate_tenors = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0])
    DEMO_CURRENCY = "USD"
    DEMO_RATE_INDEX_STUB = "IR"
    OPT_UNDERLYING_SYMBOL = "DEMO_OPT_STOCK"
    CONV_UNDERLYING_SYMBOL = "DEMO_CONV_STOCK"

    default_g2_p = (0.01, 0.003, 0.015, 0.006, -0.75) if use_hardcoded_g2_params else None
    if not use_hardcoded_g2_params:
        print("Warning: G2 calibration is not performed in this script. Using default_g2_p=None if hardcoded is False.")

    base_demo_currency_rates_values = np.array([0.020, 0.021, 0.022, 0.025, 0.027, 0.030, 0.032, 0.033])
    base_rates_map = {f"{DEMO_CURRENCY}_{DEMO_RATE_INDEX_STUB}_{t:.2f}Y": base_demo_currency_rates_values[i] for i, t in enumerate(numeric_rate_tenors)}

    opt_s0_factor = f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_S0"
    opt_vol_factor = f"{DEMO_CURRENCY}_{OPT_UNDERLYING_SYMBOL}_VOL"
    conv_s0_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_S0"
    conv_vol_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_EQVOL"
    conv_div_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_DIVYIELD"
    conv_cs_factor = f"{DEMO_CURRENCY}_{CONV_UNDERLYING_SYMBOL}_CS"

    base_s0_map = {opt_s0_factor: 100.0, conv_s0_factor: 100.0}
    base_vol_map = {opt_vol_factor: 0.25, conv_vol_factor: 0.25}
    base_other_factors_map_for_conv = {conv_div_factor: 0.01, conv_cs_factor: 0.015}
    temp_s0_map_for_gen = {**base_s0_map, **base_other_factors_map_for_conv}
    temp_vol_map_for_gen = {**base_vol_map}

    scenario_gen_global = SimpleRandomScenarioGenerator(
        base_rates_map=base_rates_map, base_s0_map=temp_s0_map_for_gen,
        base_vol_map=temp_vol_map_for_gen, random_seed=42)

    benchmark_scenarios, benchmark_factor_names = scenario_gen_global.generate_scenarios(n_benchmark_scenarios)
    print(f"Generated {n_benchmark_scenarios} benchmark scenarios with {len(benchmark_factor_names)} factors.")

    ql.Settings.instance().evaluationDate = ql.Date(val_d.day, val_d.month, val_d.year)
    parallel_workers = os.cpu_count() if enable_parallel_tff_fitting else False
    if enable_parallel_tff_fitting and parallel_workers is None: parallel_workers = 1


    # --- Instrument Definitions and Pricer Templates ---
    vanilla_static = QuantLibBondStaticBase.from_dict({
        'valuation_date': val_d, 'maturity_date': (val_d + relativedelta(years=5)),
        'coupon_rate': 0.03, 'face_value': 100.0, 'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB, 'freq': 2})
    ql_pricer_vanilla = QuantLibBondPricer(vanilla_static, method='discount')

    callable_static = CallableBondStaticBase.from_dict({
        'valuation_date': val_d, 'maturity_date': (val_d + relativedelta(years=5)),
        'coupon_rate': 0.032, 'face_value': 100.0, 'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB, 'freq': 2,
        'call_dates': [(val_d + relativedelta(years=y)) for y in [2,3,4]],
        'call_prices': [102.0, 101.0, 100.0]})
    ql_pricer_callable = QuantLibBondPricer(callable_static, method='g2', grid_steps=32)

    conv_static_s0_dynamic = ConvertibleBondStaticBase.from_dict({
        'valuation_date': val_d, 'issue_date': (val_d - relativedelta(months=6)),
        'maturity_date': (val_d + relativedelta(years=5, months=-6)), 'coupon_rate': 0.02,
        'conversion_ratio': 20.0, 'face_value': 100.0, 'currency': DEMO_CURRENCY,
        'index_stub': DEMO_RATE_INDEX_STUB, 'underlying_symbol': CONV_UNDERLYING_SYMBOL, 'freq': 2})
    ql_pricer_conv_s0_dynamic = QuantLibBondPricer(conv_static_s0_dynamic, method='convertible_binomial', convertible_engine_steps=50)
    conv_s0_dynamic_fixed_params = {'dividend_yield': 0.01, 'equity_volatility': 0.25, 'credit_spread': 0.015}

    option_static = EuropeanOptionStatic.from_dict({
        'valuation_date': val_d, 'expiry_date': (val_d + relativedelta(years=1)),
        'strike_price': 105.0, 'option_type': 'call',
        'currency': DEMO_CURRENCY, 'underlying_symbol': OPT_UNDERLYING_SYMBOL})
    bs_pricer_option = BlackScholesPricer(option_static, risk_free_rate=0.025, dividend_yield=0.01)

    instrument_setups = [
        {"id": "Vanilla_Bond_QL", "static": vanilla_static, "full_pricer": ql_pricer_vanilla,
         "tff_config_kwargs": {"bond_numeric_tenors": numeric_rate_tenors}},
        {"id": "Callable_Bond_G2", "static": callable_static, "full_pricer": ql_pricer_callable,
         "tff_config_kwargs": {"bond_numeric_tenors": numeric_rate_tenors},
         "tff_training_fixed_params": {"g2_params": default_g2_p},
         "full_pricer_kwargs": {"g2_params": default_g2_p}
        },
        {"id": "Convertible_S0_Dynamic", "static": conv_static_s0_dynamic, "full_pricer": ql_pricer_conv_s0_dynamic,
         "tff_config_kwargs": {"bond_numeric_tenors": numeric_rate_tenors, "convertible_tff_market_inputs_as_factors": False},
         "tff_training_fixed_params": conv_s0_dynamic_fixed_params,
         "full_pricer_kwargs": conv_s0_dynamic_fixed_params
        },
        {"id": "European_Option_BS", "static": option_static, "full_pricer": bs_pricer_option,
         "tff_config_kwargs": {}
        },
    ]

    print("\n--- Calibrating TFF Models ---")
    fitted_tffs = {}

    for setup in instrument_setups:
        instrument_id = setup['id']
        print(f"  Calibrating TFF for: {instrument_id}")
        tff_cal = None
        fit_time = -1.0
        try:
            if instrument_id == "Callable_Bond_G2" and setup.get("tff_training_fixed_params", {}).get("g2_params") is None:
                print(f"    WARNING: g2_params are None for {instrument_id}. Skipping TFF calibration.")
                continue

            tff_cal = TensorFunctionalFormCalibrate(
                pricer_template=setup['full_pricer'],
                scenario_generator=scenario_gen_global,
                **setup.get("tff_config_kwargs", {})
            )

            # Generate scenarios specifically for the factors this TFF needs for domain definition
            # This is used by sample_and_fit to scale the Sobol/LHS samples.
            scens_for_this_tff_domain, _ = scenario_gen_global.generate_scenarios(
                num_scenarios=100, # A small number just to define the domain boundaries
                target_factor_names=tff_cal.tff_input_raw_factor_names
            )
            if scens_for_this_tff_domain.size == 0 and tff_cal.tff_input_raw_factor_names:
                print(f"    WARNING: Could not generate domain scenarios for {instrument_id}. Skipping TFF.")
                continue

            start_calibration_time = time.time()
            model, _, _, rmse, norm_params = tff_cal.sample_and_fit(
                full_market_scenarios_for_tff_factors=scens_for_this_tff_domain,
                n_train=n_tff_train_samples,
                n_test=n_tff_test_samples,
                random_seed=42,
                parallel_workers=parallel_workers,
                option_feature_order=option_feature_eng_order if instrument_id == "European_Option_BS" else 0,
                **(setup.get("tff_training_fixed_params", {}))
            )
            fit_time = time.time() - start_calibration_time

            if model and norm_params:
                fitted_tffs[instrument_id] = {
                    "model": model,
                    "norm_params": norm_params,
                    "raw_input_names": tff_cal.tff_input_raw_factor_names,
                    "option_feature_order": option_feature_eng_order if instrument_id == "European_Option_BS" else 0,
                    "rmse": rmse,
                    "calibration_time": fit_time
                }
                print(f"    TFF for {instrument_id} calibrated. RMSE: {rmse:.6f}, Calib Time: {fit_time:.2f}s")
            else:
                print(f"    WARNING: TFF fitting failed for {instrument_id}.")
        except Exception as e:
            print(f"    ERROR calibrating TFF for {instrument_id}: {e}")
            import traceback
            traceback.print_exc()


    print("\n--- Performance Benchmarking ---")
    results_summary = []

    for setup in instrument_setups:
        instrument_id = setup['id']
        print(f"  Benchmarking: {instrument_id}")
        full_pricer = setup['full_pricer']

        tff_data = fitted_tffs.get(instrument_id)
        if not tff_data or tff_data.get("model") is None:
            print(f"    Skipping {instrument_id} as TFF model was not successfully calibrated.")
            continue

        tff_model = tff_data['model']
        tff_norm_params = tff_data['norm_params']
        tff_raw_input_names = tff_data['raw_input_names']
        tff_opt_order = tff_data['option_feature_order']
        tff_calib_time = tff_data['calibration_time']

        scenarios_for_this_instrument_raw = get_scenario_slice(benchmark_scenarios, benchmark_factor_names, tff_raw_input_names)
        if scenarios_for_this_instrument_raw.size == 0 and tff_raw_input_names:
            print(f"    Could not slice benchmark scenarios for {instrument_id} with factors {tff_raw_input_names}. Skipping.")
            continue

        start_time_full = time.time()
        full_pricer_call_kwargs = setup.get("full_pricer_kwargs", {})

        if isinstance(setup['static'], EuropeanOptionStatic):
            if scenarios_for_this_instrument_raw.shape[1] == 2:
                s0_scens = scenarios_for_this_instrument_raw[:, 0]
                vol_scens = scenarios_for_this_instrument_raw[:, 1]
                full_prices = full_pricer.price(
                    stock_price=s0_scens,
                    volatility=vol_scens,
                    **full_pricer_call_kwargs # Pass other kwargs if BS pricer uses them
                )
            else:
                print(f"    WARNING: Option scenario data for full pricer has incorrect shape {scenarios_for_this_instrument_raw.shape}. Skipping benchmark.")
                continue
        else: # For bonds
            full_prices = full_pricer.price(
                pillar_times=numeric_rate_tenors,
                market_scenario_data=scenarios_for_this_instrument_raw,
                **full_pricer_call_kwargs
            )
        time_full = time.time() - start_time_full

        tff_inputs_for_eval = scenarios_for_this_instrument_raw
        if isinstance(setup['static'], EuropeanOptionStatic) and tff_norm_params.get('is_engineered', False):
            if scenarios_for_this_instrument_raw.shape[1] != 2:
                 print(f"    Skipping TFF eval for {instrument_id}: Option requires 2 raw inputs for engineering, got {scenarios_for_this_instrument_raw.shape[1]}")
                 continue
            eng_feat, _ = engineer_option_features(scenarios_for_this_instrument_raw[:,0], scenarios_for_this_instrument_raw[:,1], order=tff_opt_order)
            np_means = np.array(tff_norm_params['means']) if tff_norm_params.get('means') is not None else None
            np_stds = np.array(tff_norm_params['stds']) if tff_norm_params.get('stds') is not None else None
            tff_inputs_for_eval, _, _ = normalize_features(eng_feat, np_means, np_stds)

        start_time_tff_eval = time.time()
        tff_prices = tff_model(tff_inputs_for_eval)
        time_tff_eval = time.time() - start_time_tff_eval # Corrected variable name

        speedup_eval_only = time_full / time_tff_eval if time_tff_eval > 1e-9 else float('inf')
        total_tff_time = tff_calib_time + time_tff_eval
        speedup_total = time_full / total_tff_time if total_tff_time > 1e-9 else float('inf')

        results_summary.append({
            "Instrument": instrument_id,
            "Full Pricer Time (s)": f"{time_full:.4f}",
            "TFF Calib Time (s)": f"{tff_calib_time:.2f}",
            "TFF Eval Time (s)": f"{time_tff_eval:.4f}", # Corrected variable name
            "Speedup (Eval Only)": f"{speedup_eval_only:.1f}x",
            "Speedup (Total TFF)": f"{speedup_total:.1f}x", # Added total speedup
            "TFF RMSE": f"{tff_data['rmse']:.6f}"
        })
        print(f"    {instrument_id}: Full Time={time_full:.4f}s, TFF Calib={tff_calib_time:.2f}s, TFF Eval={time_tff_eval:.4f}s, Speedup(Eval)={speedup_eval_only:.1f}x, Speedup(Total)={speedup_total:.1f}x, RMSE={tff_data['rmse']:.6f}")


    print("\n--- Performance Summary ---")
    if results_summary:
        header = results_summary[0].keys()
        header_str_lengths = [len(str(h)) for h in header]
        row_str_lengths = []
        for row in results_summary:
            row_str_lengths.append([len(str(v)) for v in row.values()])

        max_lengths = header_str_lengths
        for r_lengths in row_str_lengths:
            max_lengths = [max(m, l) for m, l in zip(max_lengths, r_lengths)]

        header_fmt = " | ".join([f"{{:<{max_lengths[i]}}}" for i in range(len(header))])
        separator_fmt = "-|-".join(['-' * max_lengths[i] for i in range(len(header))])

        print(header_fmt.format(*header))
        print(separator_fmt)
        for row in results_summary:
            print(header_fmt.format(*row.values()))
    else:
        print("No TFF models were successfully benchmarked.")

    print("\n--- End of Performance Demonstration ---")


if __name__ == "__main__":
    try:
        print(f"QuantLib version: {ql.__version__}")
        run_performance_demonstration(
            n_benchmark_scenarios=1028 ,  # Updated as per user request
            n_tff_train_samples=64,     # Updated as per user request
            n_tff_test_samples=8,       # Updated as per user request
            enable_parallel_tff_fitting=False,
            option_feature_eng_order=3
        )
    except NameError as e:
        if any(cn in str(e) for cn in ['ProductStaticBase','TensorFunctionalFormCalibrate']):
            print(f"ERROR: Class not defined. Ensure all notebook cells for class definitions are executed. Details: {e}")
        else: print(f"A NameError: {e}"); import traceback; traceback.print_exc()
    except Exception as e:
        print(f"An unexpected error: {e}"); import traceback; traceback.print_exc()


QuantLib version: 1.38
--- TFF Performance Demonstration ---
Benchmarking with 1028 scenarios.
TFF training with 64 samples, testing with 8 samples.
Generated 1028 benchmark scenarios with 14 factors.

--- Calibrating TFF Models ---
  Calibrating TFF for: Vanilla_Bond_QL
   Generating 64 training prices sequentially...
   Generating 8 test prices sequentially...
    TFF for Vanilla_Bond_QL calibrated. RMSE: 0.000003, Calib Time: 0.02s
  Calibrating TFF for: Callable_Bond_G2
   Generating 64 training prices sequentially...
   Generating 8 test prices sequentially...
    TFF for Callable_Bond_G2 calibrated. RMSE: 0.010235, Calib Time: 1.50s
  Calibrating TFF for: Convertible_S0_Dynamic
   Generating 64 training prices sequentially...
   Generating 8 test prices sequentially...
    TFF for Convertible_S0_Dynamic calibrated. RMSE: 0.000001, Calib Time: 0.03s
  Calibrating TFF for: European_Option_BS
   Generating 64 training prices sequentially...
   Engineering features for option TFF (or